## Konfiguracja

In [158]:
from google.colab import userdata

In [159]:
%cd /
from google.colab import drive
drive.mount('/content/gdrive/')

!ln -s /content/gdrive/My\ Drive/ ./mydrive

%cd /content/gdrive/MyDrive/projekt_inzynierski

/
Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
ln: failed to create symbolic link './mydrive/My Drive': File exists
/content/gdrive/MyDrive/projekt_inzynierski


In [160]:
!git config --global user.email "klaudiacayen@gmail.com"
!git config --global user.name "klaudia-pruchnik"

# TOKEN = userdata.get('TOKEN')
# REPO_URL = "https://github.com/klaudia-pruchnik/Proptify"
# !git clone https://{TOKEN}@{REPO_URL.replace('https://', '')}

REPO_NAME = "Proptify"
%cd /content/gdrive/MyDrive/projekt_inzynierski/{REPO_NAME}

!git add .
!git status

/content/gdrive/MyDrive/projekt_inzynierski/Proptify
On branch main
Your branch is up to date with 'origin/main'.

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	modified:   algoritm_near_finished.ipynb



In [161]:
!pip install sentence-transformers -q
!pip install -q faiss-cpu sentence-transformers
!python -m spacy download en_core_web_sm
!python -m spacy download pl_core_news_sm
!pip install gliner
!python -m spacy download en_core_web_md
!python -m spacy download pl_core_news_lg
!pip install spacy gliner transformers torch
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 48.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 34.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pl_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 29.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('

In [162]:
import faiss
import torch
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from collections import defaultdict
from math import log
from numpy.linalg import norm
import re
from gliner import GLiNER
import spacy
from spacy.matcher import Matcher
from spacy.util import filter_spans
from sklearn.metrics.pairwise import cosine_similarity
import torch.nn.functional as F
from langdetect import detect, DetectorFactory

Modele

In [163]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_e5 = SentenceTransformer('intfloat/multilingual-e5-base', device=device)
model_gliner = GLiNER.from_pretrained("urchade/gliner_multi-v2.1")
nlp_pl = spacy.load("pl_core_news_lg")
nlp_en = spacy.load("en_core_web_md")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:566: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Dataframy

In [164]:
df_songs = pd.read_parquet("../df_full_with_embeddings.parquet")

## Data preparation dodatkowe

zamień tag_list na listę

In [165]:
def _ensure_list(x):
    if isinstance(x, list): return x
    if pd.isna(x) or x is None: return []
    return [t.strip() for t in str(x).split(",") if t.strip()]

df_songs["tags_list"] = df_songs["tags"].apply(_ensure_list)
df_songs["tag_count"] = df_songs["tags_list"].apply(len)
df_songs.head(5)

,track_id,name,artist,spotify_preview_url,tags,genre,year,duration_ms,danceability,energy,...,spotify_url,explicit,album_images,spotify_id,n_tempo,n_loudness,tags_list,tags_count,tags_embedding,tag_count
0,TRIOREW128F424EAF0,Mr. Brightside,The Killers,https://p.scdn.co/mp3-preview/4d26180e6961fd46...,"rock, alternative, indie, alternative_rock, in...",None,2004,222200,0.355,0.918,...,https://open.spotify.com/track/003vvx7Niy0yvhv...,False,"[{""height"": 640, ""width"": 640, ""url"": ""https:/...",003vvx7Niy0yvhvHt4a68B,0.619996,0.874265,"[rock, alternative, indie, alternative_rock, i...",6,"[-0.005010171327739954, 0.01898571476340294, -...",6
1,TRRIVDJ128F429B0E8,Wonderwall,Oasis,https://p.scdn.co/mp3-preview/d012e536916c927b...,"rock, alternative, indie, pop, alternative_roc...",None,2006,258613,0.409,0.892,...,https://open.spotify.com/track/5qqabIl2vWzo9Ap...,False,"[{""height"": 640, ""width"": 640, ""url"": ""https:/...",5qqabIl2vWzo9ApSC317sa,0.730137,0.874061,"[rock, alternative, indie, pop, alternative_ro...",9,"[0.003282089252024889, 0.02081618271768093, -0...",9
2,TROUVHL128F426C441,Come as You Are,Nirvana,https://p.scdn.co/mp3-preview/a1c11bb1cb231031...,"rock, alternative, alternative_rock, 90s, grunge",RnB,1991,218920,0.508,0.826,...,https://open.spotify.com/track/2RsAajgo0g7bMCH...,False,"[{""height"": 640, ""width"": 640, ""url"": ""https:/...",2RsAajgo0g7bMCHxwH3Sk0,0.502363,0.851906,"[rock, alternative, alternative_rock, 90s, gru...",5,"[-0.000298114464385435, 0.017946673557162285, ...",5
3,TRUEIND128F93038C4,Take Me Out,Franz Ferdinand,https://p.scdn.co/mp3-preview/399c401370438be4...,"rock, alternative, indie, alternative_rock, in...",None,2004,237026,0.279,0.664,...,https://open.spotify.com/track/20I8RduZC2PWMWT...,False,"[{""height"": 640, ""width"": 640, ""url"": ""https:/...",20I8RduZC2PWMWTDCZuuAN,0.437682,0.803699,"[rock, alternative, indie, alternative_rock, i...",8,"[0.0032091771718114614, 0.018290618434548378, ...",8
4,TRLNZBD128F935E4D8,Creep,Radiohead,https://p.scdn.co/mp3-preview/e7eb60e9466bc3a2...,"rock, alternative, indie, alternative_rock, in...",RnB,2008,238640,0.515,0.430,...,https://open.spotify.com/track/70LcF31zb1H0PyJ...,True,"[{""height"": 640, ""width"": 640, ""url"": ""https:/...",70LcF31zb1H0PyJoS1Sx1r,0.384441,0.786666,"[rock, alternative, indie, alternative_rock, i...",8,"[0.007342000957578421, 0.020590929314494133, -...",8


Uzupełnij tagi ich tagami nadrzędnymi dla lepszego dopasowania (to trzeba przenieść do innego pliku i poprawić bazę)

In [166]:
# reguły: subgatunek -> nadrzędny
ALSO_ADD_PARENT = {
    # rock
    "progressive rock": "rock",
    "classic rock": "rock",
    "indie rock": "rock",
    "hard rock": "rock",
    "pop rock": "rock",
    "psychedelic rock": "rock",
    "punk rock": "rock",
    "blues rock": "rock",
    "post rock": "rock",

    # metal
    "heavy metal": "metal",
    "death metal": "metal",
    "black metal": "metal",
    "doom metal": "metal",
    "thrash metal": "metal",
    "melodic death metal": "metal",
    "symphonic metal": "metal",
    "gothic metal": "metal",
    "nu metal": "metal",
    "progressive metal": "metal",
    "power metal": "metal",
    "metalcore": "metal",

    # pop / electronic
    "indie pop": "pop",
    "synthpop": "pop",
    "drum and bass": "electronic",
}

# 2) reguły: słowa-klucze → nadrzędny
PARENT_KEYWORDS = {
    "rock": ["rock"],
    "metal": ["metal"],
    "pop": ["pop", "britpop"],
    "hip hop": ["hip hop", "rap"],
    "electronic": ["electronic", "techno", "house", "trance", "idm", "downtempo", "electro", "ambient"],
    "jazz": ["jazz"],
    "classical": ["classical"],
    "punk": ["punk"],
    "folk": ["folk"],
    "blues": ["blues"],
    "country": ["country"],
    "reggae": ["reggae"],
}

def expand_tags(tag_list):
    if tag_list is None or (isinstance(tag_list, float) and pd.isna(tag_list)):
        tag_list = []

    # normalizacja minimalna: małe litery, zamiana _ → spacja
    tags = {str(t).lower().replace("_", " ").strip() for t in tag_list}

    # 1) subgatunek → nadrzędny
    for child, parent in ALSO_ADD_PARENT.items():
        if child in tags:
            tags.add(parent)

    # 2) słowa kluczowe → nadrzędny
    for parent, kws in PARENT_KEYWORDS.items():
        if any(kw in t for t in tags for kw in kws):
            tags.add(parent)

    return sorted(tags)

# zastosowanie
df_songs["tags_list"] = df_songs["tags_list"].apply(expand_tags)


In [167]:
df_songs["tags_list"] = df_songs["tags_list"].apply(
    lambda lst: [tag.replace("_", " ") for tag in lst]
)

# Algorytm dopasowania utworów

## Przygotowanie

Budowanie inverted index do późniejszego scorowania utworów

In [168]:
df_songs = df_songs.reset_index(drop=True)

def build_inverted_index(df_songs: pd.DataFrame, tags_col: str = "tags_list"):
    inv = defaultdict(list)
    df_count = defaultdict(int)  # document frequency: w ilu utworach wystąpił tag

    for i, tags in enumerate(df_songs[tags_col]):
        if not tags:
            continue
        seen = set()
        for t in tags:
            inv[t].append(i)
            if t not in seen:
                df_count[t] += 1
                seen.add(t)

    # zamiana listy na np
    for t in inv:
        inv[t] = np.asarray(inv[t], dtype=np.int32)

    return inv, df_count

INV_INDEX, DF_COUNT = build_inverted_index(df_songs, tags_col="tags_list")
N_SONGS = len(df_songs)
AVG_TAG_LEN = float(df_songs["tag_count"].mean()) if "tag_count" in df_songs else 10.0

Config do wszystkich parametrów algorytmu

In [169]:
# 1. KONFIGURACJA EKSTRAKCJI I DOPASOWANIA (NLP / E5 / GLiNER)
EXTRACTION_CONFIG = {
    "gliner_threshold": 0.3, # Próg pewności dla GLiNERa (klasyfikacja: czy to tag, czy audio?)
    "tag_similarity_threshold": 0.65, # Próg podobieństwa cosinusowego dla mapowania frazy na Tag w bazie (E5)
    "audio_confidence_threshold": 0.78, # Próg pewności dla mapowania frazy na cechę Audio (E5)
}

# 2. KONFIGURACJA PUNKTACJI (SCORING)
SCORING_CONFIG = {
    # Tagi
    "use_idf": True,            # Czy rzadsze tagi mają ważyć więcej? (Zalecane: True)
    "query_pow": 1.0,           # Wyostrzanie wag z promptu (1.0 = liniowo, >1.0 = faworyzuje główne frazy)

    # Fuzja
    "audio_weight": 0.4,        # Ile wagi ma Audio vs Tagi? (0.4 = 40% Audio, 60% Tagi)
                                # Jeśli nie znaleziono tagów, kod automatycznie ustawi 1.0 dla Audio.
}

# 3. KONFIGURACJA POSZUKIWANIA KANDYDATÓW (RETRIEVAL)
RETRIEVAL_CONFIG = {
    "n_candidates": 400,        # Ile utworów wstępnie pobieramy z bazy (top N po tagach)
    "flat_delta": 0.05,         # Soft cut-off: o ile gorszy wynik akceptujemy, by nie ucinać "na sztywno"
}

# 4. KONFIGURACJA TIERÓW I ZBIORU ROBOCZEGO (WORKING SET)
WORKSET_CONFIG = {
    # Progi jakości (Quality Gates) dla funkcji calculate_dynamic_thresholds
    "min_absolute_high": 0.75,  # Próg dla Tier A nie spadnie poniżej tego
    "min_absolute_mid": 0.50,   # Próg dla Tier B nie spadnie poniżej tego

    # Budowanie zbioru roboczego
    "target_pool_size": 100,    # Do ilu utworów chcemy dobić przed finalnym losowaniem
    "min_required_size": 15,    # Absolutne minimum (ratujemy się Tierem C, jeśli mniej)

    # Ratowanie hitów z Tieru B
    "popularity_rescue_ratio": 0.2, # 20% miejsc z Tieru B rezerwujemy dla najpopularniejszych (nawet jeśli mają niższy score)
}

# 5. KONFIGURACJA POPULARNOŚCI (BUCKETING)
POPULARITY_CONFIG = {
    # Progi podziału na koszyki
    "p_high": 70,               # Powyżej 70 to High Popularity
    "p_mid": 35,                # Powyżej 35 to Mid Popularity

    # Docelowy miks w finalnej playliście (musi sumować się do 1.0)
    "mix": {
        "high": 0.40,           # 40% Hitów
        "mid":  0.35,           # 35% Solidnych średniaków
        "low":  0.25,           # 25% Niszowych odkryć
    },

    # Kotwice (Forced Popular)
    "forced_popular": 2,        # Ile super-hitów wrzucić na start (gwarantowane)
    "forced_popular_min": 80,   # Jaka popularność definiuje super-hit do sekcji forced
}

# 6. KONFIGURACJA FINALNEGO LOSOWANIA (SAMPLING)
SAMPLING_CONFIG = {
    "final_n": 15,              # Długość playlisty
    "alpha": 2.0,               # Temperatura losowania (Weighted Sample).
                                # 2.0 = mocno faworyzuje utwory z wysokim score (pasujące).
                                # 0.0 = losowanie całkowicie losowe (ignoruje score).
    "shuffle": True,            # Czy pomieszać kolejność na końcu? (False = sortowanie po score, do debugu)
}

zadanie: przeanalizować to jakie wartości ustawić, od ilu sie zaczyna np szybka muzyka itp.

## Wyciąganie relevant phrases z prompta

In [170]:
# Ustawiamy ziarno dla detekcji języka (żeby wyniki były powtarzalne dla krótkich tekstów)
DetectorFactory.seed = 0

In [171]:
# spaCy: Model do parsowania gramatycznego i Matchera (zapewnia precyzyjne wzorce)
nlp_pl = spacy.load("pl_core_news_lg")
nlp_en = spacy.load("en_core_web_md")

In [172]:
GENERIC_LEMMAS = [
    # Polski
    "muzyka", "utwór", "piosenka", "kawałek", "playlista", "lista", "numer", "rok", "klimat", "styl",
    # Angielski
    "music", "song", "track", "playlist", "list", "number", "vibe", "tune", "genre", "style"
]

GENERIC_VERBS = [
    # --- POLSKI (Bezokoliczniki / Lematy) ---
    # Szukanie / Chcenie
    "szukać", "poszukiwać", "chcieć", "pragnąć", "potrzebować", "woleć", "wymagać",
    # Bycie / Posiadanie
    "być", "mieć", "znajdować", "znaleźć",
    # Słuchanie / Odtwarzanie
    "słuchać", "posłuchać", "usłyszeć", "grać", "zagrać", "puszczać", "puścić", "odtworzyć", "zapodać",
    # Prośby / Rekomendacje
    "prosić", "polecić", "polecać", "rekomendować", "sugerować", "zaproponować", "dawać", "dać",

    # --- ANGIELSKI (Base forms) ---
    # Searching / Wanting
    "search", "look", "find", "want", "need", "desire", "wish", "require",
    # Being / Having
    "be", "have", "get",
    # Listening / Playing
    "listen", "hear", "play", "replay", "stream",
    # Requests
    "give", "recommend", "suggest", "show", "provide",
]

NEGATION_TERMS = [
    # PL
    "nie", "bez", "mało", "zero", "ani", "żaden", "brak", "mniej",
    # EN
    "no", "not", "without", "less", "non", "neither", "nor", "lack", "zero"
]

In [173]:
def create_matcher_for_nlp(nlp_instance):
    """Tworzy obiekt Matcher przypisany do konkretnego modelu językowego"""
    matcher = Matcher(nlp_instance.vocab)

    # Warunek wykluczający
    # To musi być Rzeczownik, ale nie może być na liście generycznej
    noun_filter = {
        "POS": {"IN": ["NOUN", "PROPN"]},
        "IS_STOP": False,
        "LEMMA": {"NOT_IN": GENERIC_LEMMAS}
    }

    matcher.add("FRAZA", [
        # 1. Samodzielny rzeczownik lub nazwa własna
        # Wyłapuje pojedyncze słowa kluczowe, np. "rock", "jazz", "Metallica".
        [noun_filter],

        # 2. Przymiotnik + Rzeczownik
        # Klasyczna fraza opisująca cechę obiektu, np. "szybki bas", "ciężkie brzmienie".
        [{"POS": "ADJ"}, noun_filter],

        # 3. Wyrażenie przyimkowe (opcjonalnie poprzedzone przysłówkiem)
        # Określa przeznaczenie, styl lub pochodzenie, np. "do tańca", "z klimatem", "prosto z serca".
        [{"POS": "ADV", "OP": "?"}, {"POS": "ADP"}, noun_filter],

        # 4. Przysłówek + Przymiotnik
        # Służy do wzmocnienia lub doprecyzowania cechy, np. "bardzo wesoła", "niezwykle głośny".
        [{"POS": "ADV"}, {"POS": "ADJ", "IS_STOP": False}],

        # 5. Samodzielny przymiotnik
        # Gdy użytkownik używa samej cechy bez rzeczownika (częste w mowie potocznej), np. "rockowa", "spokojne".
        [{"POS": "ADJ", "IS_STOP": False}],

        # 6. Rzeczowniki złożone (Dwa rzeczowniki obok siebie)
        # Wyłapuje gatunki lub nazwy dwuczłonowe, np. "post rock", "hip hop", "death metal".
        [{"POS": {"IN": ["NOUN", "PROPN"]}, "IS_STOP": False}, noun_filter],

        # 7. Rozbudowana fraza przymiotnikowa z przyimkiem
        # Opisuje przydatność lub relację, np. "dobra do tańca", "idealny na imprezę".
        [{"POS": "ADV", "OP": "?"}, {"POS": "ADJ"}, {"POS": "ADP"}, noun_filter],

        # 8. Fraza czasownikowa (Czasownik + Dopełnienie)
        # Wyklucza ogólne czasowniki
        [
            {"POS": "VERB", "LEMMA": {"NOT_IN": GENERIC_VERBS}},
            {"POS": {"IN": ["NOUN", "ADJ", "PRON"]}, "OP": "+"} # Dopełnienie (może składać się z kilku słów)
        ]
    ])
    return matcher

# Tworzymy matchery raz na starcie
matcher_pl = create_matcher_for_nlp(nlp_pl)
matcher_en = create_matcher_for_nlp(nlp_en)


In [174]:
# 4. FUNKCJA POMOCNICZA: SPRAWDZANIE NEGACJI

def is_span_negated(doc, start_index, window=2):
    """
    Sprawdza, czy przed frazą (start_index) stoi słowo przeczące.
    Patrzy 'window' tokenów wstecz.
    """
    lookback = max(0, start_index - window)
    preceding_tokens = doc[lookback:start_index]

    for token in preceding_tokens:
        if token.text.lower() in NEGATION_TERMS:
            return True
    return False

In [175]:
# GŁÓWNA FUNKCJA EKSTRAKCJI

def extract_relevant_phrases(prompt):
    # Setup językowy
    prompt = prompt.lower()
    prompt_clean = prompt.strip()

    if not prompt_clean:
        return []

    try:
        lang_code = detect(prompt)
    except:
        lang_code = 'pl'

    if lang_code == 'en':
        current_nlp = nlp_en
        current_matcher = matcher_en
        lang_msg = "EN"
    else:
        current_nlp = nlp_pl
        current_matcher = matcher_pl
        lang_msg = "PL"

    doc = current_nlp(prompt)

    # Matcher
    matcher_matches = current_matcher(doc)

    # Konwersja matchy na obiekty Span
    matcher_spans = [doc[start:end] for match_id, start, end in matcher_matches]

    # filter_spans usuwa nakładające się frazy (np. "rock" wewnątrz "post rock")
    # zostawiając najdłuższe dopasowanie
    combined_spans = filter_spans(matcher_spans)

    final_phrases = []

    for span in combined_spans:
        # Sprawdzanie negacji
        if is_span_negated(doc, span.start):
            continue

        final_phrases.append(span.text.lower())

    # Czyszczenie i sortowanie wyników
    unique_phrases = sorted(list(set([p.strip() for p in final_phrases if len(p.strip()) > 2])))

    print(f"[{lang_msg}] Prompt: '{prompt}' \n-> {unique_phrases}")

    return unique_phrases

In [211]:
# ----------------------------------------------------------------------
# PRZYKŁADY UŻYCIA
# ----------------------------------------------------------------------

TEST_PROMPT_1 = "szukam muzyki rockowej, ale takiej pełnej spokoju i bardzo wesołej, trochę do tańca"
TEST_PROMPT_2 = "rock, pop, dance i coś do tańca, zależy mi na maksymalnej energii"
TEST_PROMPT_3 = "rock, pop, dance i coś do tańca, zależy mi na maksymalnej energii, post rock, alternative rock, rock alternatywny, pop"
TEST_PROMPT_4 = "muzyka rockowa z lat 90, z klimatem podróży, postpankowa"
TEST_PROMPT_5 = "muzyka bez słów, smutna, nostalgiczna, spokojna"
TEST_PROMPT_6 = "szybka, intensywna, bardzo dobra do tańca, zabawy, zajsta do tańca, idealna do tańca, "
TEST_PROMPT_7 = "albo zwykły rock, albo jakiś post rock albo punk, coś takiego"
TEST_PROMPT_8 = "muzyka dynamiczna, szybko, szybkie tempo, wysokie tempo, energiczna"
TEST_PROMPT_9 = "muzyka, która koi nerwy"
TEST_PROMPT_10 = "muzyka, która koi nerwy"
TEST_PROMPT_11 = "Szukam muzyki rockowej, ale nie smutnej"
TEST_PROMPT_12 = "I want energetic songs, no slow music"
TEST_PROMPT_13 = "Chcę wesołe piosenki, coś szybkiego, ruchliwego, rock, muzyka popowa, z lat 90, dużo gitary, shoegaze, "
# TEST_PROMPT_14 = "Vaporwave, J-Core (Japanese Hardcore), Blackened Death-Doom, Skweee, Dungeon Synth, Breakcore, Mathcore, Drone Metal (Ambient Drone), Folktronica, Lowercase"
TEST_PROMPT_14 = "szybki rock"

picked_prompt = 'szybka'

relevant_phrases = extract_relevant_phrases(picked_prompt)
# extract_and_route(TEST_PROMPT_4)

[PL] Prompt: 'szybka' 
-> ['szybka']


## Dzielimy znalezione frazy na audio i tag

In [212]:
# ACTIVITY_GROUPS = {
#     'deep_focus': {
#         'triggers': [
#             "reading", "reading books", "studying", "learning", "homework",
#             "focus", "concentration", "deep work", "coding", "programming",
#             "writing", "library", "chess", "brainstorming", "thinking",
#             "exam preparation", "working", "office work", "study session"
#         ],
#         'rules': [
#             ('instrumentalness', (0.8, 1.0)),
#             ('speechiness', (0.0, 0.2)),
#             ('energy', (0.1, 0.5)),
#             ('n_loudness', (0.3, 0.6))
#         ]
#     },

#     'sleep_relax': {
#         'triggers': [
#             "sleeping", "falling asleep", "insomnia", "nap", "napping",
#             "meditation", "meditating", "yoga", "mindfulness", "zen",
#             "spa", "massage", "calm down", "anxiety relief", "stress relief",
#             "lying in bed", "winding down", "evening relaxation", "chill out"
#         ],
#         'rules': [
#             ('energy', (0.0, 0.25)),
#             ('n_tempo', (0.0, 0.35)),
#             ('n_loudness', (0.0, 0.35)),
#             ('acousticness', (0.5, 1.0)),
#             ('valence', (0.4, 0.7))
#         ]
#     },

#     'workout_intense': {
#         'triggers': [
#             "gym", "weightlifting", "crossfit", "boxing", "kickboxing",
#             "hiit", "interval training", "sprint", "running fast", "cardio",
#             "beast mode", "motivation", "pump up", "hardcore training",
#             "powerlifting", "bodybuilding", "marathon training"
#         ],
#         'rules': [
#             ('energy', (0.8, 1.0)),
#             ('n_tempo', (0.7, 1.0)),
#             ('n_loudness', (0.7, 1.0)),
#             ('danceability', (0.5, 0.9))
#         ]
#     },

#     'commute_jogging': {
#         'triggers': [
#             "jogging", "walking", "walking the dog", "commuting", "driving",
#             "road trip", "car ride", "night drive", "highway", "bus ride",
#             "train ride", "traveling", "subway", "city walk", "bike riding",
#             "cycling"
#         ],
#         'rules': [
#             ('energy', (0.5, 0.8)),
#             ('n_tempo', (0.45, 0.7)),
#             ('valence', (0.4, 0.9))
#         ]
#     },

#     'party_club': {
#         'triggers': [
#             "party", "house party", "clubbing", "dancing", "dancefloor",
#             "friday night", "saturday night", "birthday", "celebration",
#             "drinking", "pre-game", "getting ready", "festival", "rave",
#             "disco", "summer party", "pool party"
#         ],
#         'rules': [
#             ('danceability', (0.8, 1.0)),
#             ('energy', (0.7, 1.0)),
#             ('valence', (0.6, 1.0)),
#             ('n_loudness', (0.7, 1.0))
#         ]
#     },

#     'chores_background': {
#         'triggers': [
#             "cleaning", "cleaning the house", "cooking", "kitchen",
#             "doing dishes", "gardening", "chores", "housework",
#             "morning coffee", "breakfast", "sunday morning",
#             "hanging out", "friends coming over", "dinner party", "barbecue"
#         ],
#         'rules': [
#             ('energy', (0.4, 0.7)),
#             ('valence', (0.5, 0.9)),
#             ('danceability', (0.4, 0.8)),
#             ('acousticness', (0.2, 0.8))
#         ]
#     },

#     'sad_emotional': {
#         'triggers': [
#             "sad", "crying", "depression", "depressed", "lonely",
#             "heartbreak", "breakup", "missing someone", "rainy day",
#             "melancholy", "grieving", "emotional", "moody", "nostalgia",
#             "bad day"
#         ],
#         'rules': [
#             ('valence', (0.0, 0.3)),
#             ('energy', (0.0, 0.4)),
#             ('danceability', (0.0, 0.4)),
#             ('n_tempo', (0.0, 0.4))
#         ]
#     },

#     'romance': {
#         'triggers': [
#             "date night", "romantic dinner",
#             "candlelight", "intimacy", "cuddling", "boyfriend", "girlfriend",
#             "valentine", "sexy", "seduction", "late night", "bedroom"
#         ],
#         'rules': [
#             ('n_tempo', (0.1, 0.5)),
#             ('danceability', (0.5, 0.8)),
#             ('energy', (0.2, 0.6)),
#             ('n_loudness', (0.2, 0.6))
#         ]
#     },

#     'gaming': {
#         'triggers': [
#             "gaming", "playing games", "esports", "streaming", "twitch",
#             "league of legends", "fortnite", "fps", "rpg", "cyberpunk",
#             "hacker", "futuristic"
#         ],
#         'rules': [
#             ('energy', (0.7, 1.0)),
#             ('acousticness', (0.0, 0.2)),
#             ('speechiness', (0.0, 0.3))
#         ]
#     }
# }

In [213]:
# KONFIGURACJA DO KLASYFIKACJI FRAZ

# GLiNER ma odróżnić Tag (np. rock) od cech audio (np. szybka).

LABELS_CONFIG = {
    # --- TAGI ---
    "gatunek_muzyczny": {
        "desc": "rock, pop, jazz, hip hop, metal, indie, alternative, emo, psychedelic, industrial, grunge, punk, folk, electronic, experimental, noise music",
        "route": "TAGS"
    },
    "klimat_styl": {
        "desc": "chill, chillout, mellow, ambient, lounge, dark, beautiful, love",
        "route": "TAGS"
    },
    "typ_utworu": {
        "desc": "soundtrack, ost, muzyka filmowa, ścieżka dźwiękowa, remix",
        "route": "TAGS"
    },
    "instrument": {
        "desc": "piano, guitar, drums, violin, bass, saxophone, synthesizer, vocals",
        "route": "TAGS"
    },
    "okres_czasu": {
        "desc": "80s, 90s, 00s, 2020s, oldies, retro, klasyk, lata 90, lata 80",
        "route": "TAGS"
    },
    "pochodzenie": {
        "desc": "polish, american, british, french, k-pop, latino, spanish, deutsch",
        "route": "TAGS"
    },

    # --- WSZYSTKO INNE (dane audio) ---
    # Wrzucamy wszystko co jest opisem tutaj.
    "cecha_audio": {
        "desc": "sad, happy, fast, slow, danceable, party, energetic, calm, relaxing, loud, quiet, acoustic, electronic, melancholic, gloomy, euphoric, club banger",
        "route": "AUDIO"
    }
}

# Generowanie zmiennych
GLINER_LABELS = [f"{k} ({v['desc']})" for k, v in LABELS_CONFIG.items()]
ROUTING_MAP = {k: v['route'] for k, v in LABELS_CONFIG.items()}

In [214]:
def get_label_config_lists(config):
    """
    Przetwarza konfigurację etykiet, tworząc listę sformatowanych stringów dla modelu GLiNER
    oraz mapę powrotną do oryginalnych kluczy.

    Args:
        config (Dict[str, Dict[str, str]]): Słownik konfiguracyjny, w którym:
            - Kluczem (str) jest nazwa kategorii (np. 'gatunek_muzyczny').
            - Wartością (dict) jest słownik zawierający co najmniej klucz 'desc'
              z opisem słownym (np. 'rock, pop...').

    Returns:
        Tuple[List[str], Dict[str, str]]: Krotka zawierająca dwa elementy:
            1. gliner_labels (List[str]): Lista sformatowanych etykiet w postaci "Klucz (Opis)".
               Przykład: ["gatunek_muzyczny (rock, pop...)", ...].
            2. label_mapping (Dict[str, str]): Słownik mapujący pełną etykietę z powrotem na klucz.
               Przykład: {"gatunek_muzyczny (rock, pop...)": "gatunek_muzyczny"}.
    """
    gliner_labels = []
    label_mapping = {} # Mapa {"nazwa (opis)": "nazwa"}

    for key, value in config.items():
        full_label = f"{key} ({value['desc']})"
        gliner_labels.append(full_label)
        label_mapping[full_label] = key

    return gliner_labels, label_mapping

GLINER_LABELS_LIST, GLINER_LABEL_MAP = get_label_config_lists(LABELS_CONFIG)


def classify_phrases_with_gliner(prompt, spacy_phrases, model, threshold=0.3):
    """
    Klasyfikuje frazy wykryte przez spaCy, dopasowując je do encji wykrytych przez model GLiNER
    w kontekście całego promptu.

    Funkcja łączy precyzyjne wycinanie fraz (spaCy) z rozumieniem kontekstu (GLiNER).
    Opiera się na globalnych zmiennych konfiguracyjnych: GLINER_LABELS_LIST, GLINER_LABEL_MAP oraz LABELS_CONFIG.

    Args:
        prompt (str): Pełna treść zapytania użytkownika (niezbędna dla kontekstu GLiNERa).
        spacy_phrases (List[str]): Lista fraz (noun chunks/entities) wyekstrahowanych wcześniej przez spaCy.
        model (Any): Załadowany model GLiNER (np. obiekt klasy GLiNER).
        threshold (float, optional): Próg pewności dla predykcji GLiNERa. Domyślnie 0.3.

    Returns:
        List[Dict[str, str]]: Lista słowników, gdzie każdy słownik reprezentuje sklasyfikowaną frazę:
            {
                "phrase": str (oryginalna fraza ze spaCy),
                "category": str (klucz kategorii np. 'gatunek_muzyczny' lub 'cecha_audio'),
                "route": str (typ routingu np. 'TAGS' lub 'AUDIO')
            }
    """
    if not spacy_phrases:
        return []

    # Uruchamiamy GLiNER na całym tekście
    # Dzięki temu odróżni "Rock" (gatunek) od "Szybka" (audio)
    gliner_predictions = model.predict_entities(prompt, GLINER_LABELS_LIST, threshold=threshold)

    results = []

    # Iterujemy po frazach ze spaCy i szukamy dla nich etykiety w wynikach GLiNERa
    for phrase in spacy_phrases:
        matched_category = None
        matched_route = "AUDIO" # Domyślny routing (bezpieczny fallback)

        # Normalizacja frazy spaCy do porównania
        phrase_clean = phrase.lower().strip()

        # Szukamy czy ta fraza została też znaleziona przez GLiNERa
        # Sprawdzamy czy tekst encji GLiNERa zawiera się w frazie spaCy lub odwrotnie
        best_score = 0

        for entity in gliner_predictions:
            entity_text = entity['text'].lower().strip()

            # Sprawdzenie pokrycia (overlap)
            if phrase_clean in entity_text or entity_text in phrase_clean:
                # Jeśli mamy dopasowanie, pobieramy czystą nazwę kategorii
                full_label = entity['label']
                short_key = GLINER_LABEL_MAP.get(full_label)

                if short_key:
                    matched_category = short_key
                    # Pobieramy routing z konfiguracji
                    matched_route = LABELS_CONFIG[short_key]['route']
                    break # Znaleźliśmy etykietę, idziemy do następnej frazy spaCy

        # Jeśli GLiNER nic nie znalazł dla tej frazy, ale spaCy ją wykryło
        # oznaczamy jako ogólną "cecha_audio" (w dalszych etapach E5 może ją oznaszyć jako "śmieć")
        if not matched_category:
            matched_category = "cecha_audio" # Fallback
            matched_route = "AUDIO"

        results.append({
            "phrase": phrase,
            "category": matched_category,
            "route": matched_route
        })

    return results

In [215]:
classified_phrases = classify_phrases_with_gliner(picked_prompt, relevant_phrases, model_gliner)
classified_phrases

[{'phrase': 'szybka', 'category': 'cecha_audio', 'route': 'AUDIO'}]

In [216]:
# def classify_phrases_with_e5_router(phrases, model):
#     """
#     Klasyfikuje frazy do jednej z 3 grup (TAGS, AUDIO, ACTIVITY)
#     używając porównania semantycznego E5.
#     """
#     if not phrases:
#         return []

#     # 1. Definiujemy "kotwice" semantyczne dla każdej ścieżki
#     # Im więcej synonimów, tym łatwiej modelowi trafić.
#     ROUTING_PROTOTYPES = {
#         'TAGS': "passage: gatunek muzyczny styl rok dekada instrument rock pop jazz metal hip hop lata 80 90 gitara pianino soundtrack polska",

#         'AUDIO': "passage: cecha audio brzmienie tempo nastrój szybka wolna smutna wesoła głośna cicha energiczna rytmiczna akustyczna elektroniczna",

#         'ACTIVITY': "passage: aktywność intencja czynność do spania na siłownię do biegania do nauki praca skupienie relaks medytacja impreza jazda samochodem gotowanie sprzątanie seks randka"
#     }

#     route_names = list(ROUTING_PROTOTYPES.keys())
#     prototype_texts = list(ROUTING_PROTOTYPES.values())

#     # 2. Embeddujemy wzorce (można zcache'ować, żeby nie liczyć co raz)
#     prototype_vecs = model.encode(prototype_texts, normalize_embeddings=True)

#     # 3. Embeddujemy frazy użytkownika
#     # Pamiętaj o prefiksie "query: " dla E5
#     phrase_vecs = model.encode([f"query: {p}" for p in phrases], normalize_embeddings=True)

#     # 4. Liczymy podobieństwo
#     sims = cosine_similarity(phrase_vecs, prototype_vecs)

#     results = []

#     for i, phrase in enumerate(phrases):
#         scores = sims[i]
#         best_idx = np.argmax(scores)
#         best_route = route_names[best_idx]
#         best_score = scores[best_idx]

#         print(f"📡 Router E5: '{phrase}' -> {best_route} (score: {best_score:.3f})")

#         results.append({
#             "phrase": phrase,
#             "route": best_route,
#             "confidence": float(best_score)
#         })

#     return results

In [217]:
# # Masz frazy wyciągnięte np. ze spaCy albo po prostu listę stringów
# relevant_phrases = ["do spania", "szybka", "rock"]

# # 1. ROUTING PRZEZ E5
# routed_items = classify_phrases_with_e5_router(relevant_phrases, model_e5)

# # 2. Kontenery
# tags_phrases = []
# audio_phrases = []
# activity_phrases = []

# # 3. Rozdzielanie
# for item in routed_items:
#     r = item['route']
#     p = item['phrase']

#     if r == 'TAGS':
#         tags_phrases.append(p)
#     elif r == 'AUDIO':
#         audio_phrases.append(p)
#     elif r == 'ACTIVITY':
#         activity_phrases.append(p)

# print("TAGS:", tags_phrases)
# print("AUDIO:", audio_phrases)
# print("ACTIVITY:", activity_phrases)

BŁĘDY:
* nie rozpoznaje 'z lat 90'
* nie rozponaje rzeczy jak "j-core", "death-doom" - z "-"
* szybkie vaporwave, szybki rock itp. tutaj lepiej by było to rozdzielić na 2
* rozpoznało "zależy mi" jako audio feature ??

TODO:
* można też na typ etapie podczas dzielenia na audio i tag dołożyć kategorię "trash"
* trzeba coś zrobić z wyciąganiem do czego ma służyć muzyka i zdefiniować, że np do biegania ma być energiczna, do gotowania itp, a do skupienia muzyka z np wysokim instrumentallness, raczej niską energią itp.


* 'fast, energetic': bieganie, jazda samochodem (loundness wysoki), gotowanie,
* 'focus': czytanie, pisanie, programowanie
* 'relax':
* itp.

In [218]:
def prepare_queries_for_e5_separated(classified_data, original_prompt):
    """
    Rozdziela frazy na Tagi i Audio.
    Zwraca słownik z dwoma listami: "TAGS" i "AUDIO".
    """

    return {
        "TAGS": [
            item['phrase'] for item in classified_data
            if item['route'] == 'TAGS'
        ],
        "AUDIO": [
            item['phrase'] for item in classified_data
            if item['route'] == 'AUDIO'
        ]
    }

In [219]:
e5_queries_separated = prepare_queries_for_e5_separated(classified_phrases, picked_prompt)
e5_queries_separated

{'TAGS': [], 'AUDIO': ['szybka']}

## Dopasowanie audio

In [220]:
FEATURE_DESCRIPTIONS = {
    'valence': [
        # ((Min, Max), "Opis")
        ((0.0, 0.25), "very low valence, very sad, melancholic, dark, gloomy emotional mood music"),
        ((0.25, 0.40), "low valence, bittersweet, thoughtful, introspective, moody emotional mood music"),
        # ((0.45, 0.60), "medium valence, neutral emotional mood, neither clearly happy nor clearly sad music"),
        ((0.70, 0.90), "high valence, positive, pleasant, warm, cheerful, uplifting emotional mood music"),
        ((0.90, 1.0), "very high valence, very happy, joyful, exstatic, euphoric, bright, feel-good emotional mood music")
    ],

    'danceability': [
        ((0.0, 0.25), "very low danceability, not danceable, abstract or experimental, weak or irregular rhythm music"),
        ((0.25, 0.4), "low danceability, little groove, minimal rhythm, not primarily for dancing music"),
        # ((0.50, 0.70), "medium danceability, some groove, steady rhythm music"),
        ((0.70, 0.85), "high danceability, clear beat, strong groove, good for dancing, club-oriented music"),
        ((0.85, 1.0), "very high danceability, strong groove, infectious rhythm, perfect for dancing, party, club banger music")
    ],

    'acousticness': [
        ((0.0, 0.20), "very low acousticness, fully electronic, synthetic, digital, computer-generated sound music"),
        ((0.20, 0.45), "low acousticness, mostly electronic with some subtle organic or acoustic elements music"),
        # ((0.45, 0.65), "medium acousticness, balanced mix of acoustic and electronic instruments, hybrid sound music"),
        ((0.65, 0.85), "high acousticness, mostly acoustic, organic, live instruments such as accoustic guitar or piano music"),
        ((0.85, 1.0), "very high acousticness, fully acoustic, unplugged, natural, organic instruments only music")
    ],

    'n_tempo': [
        ((0.0, 0.20), "very slow tempo, very slow pace, dragging rhythm music"),
        ((0.20, 0.45), "slow tempo, downtempo, slow pace, relaxed rhythm music"),
        ((0.45, 0.7), "medium tempo, moderate pace, walking pace music"),
        ((0.80, 0.90), "fast tempo, uptempo, quick pace, energetic rhythm music"),
        ((0.90, 1.0), "very fast tempo, rapid pace, racing rhythm, frantic speed music")
    ],

    'instrumentalness': [
        ((0.0, 0.35), "very low instrumentalness, strong presence of vocals and lyrics, clear singing, vocal-focused track"),
        ((0.35, 0.75), "medium instrumentalness, mix of vocals and instrumental sections, vocals present but not constant"),
        ((0.75, 1.0), "very high instrumentalness, fully instrumental track, no vocals, no singing, music without lyrics")
    ],

    'energy': [
        ((0.0, 0.25), "very low energy, motionless, static, sleep-inducing, minimal activity music"),
        ((0.25, 0.45), "low energy, relaxed, laid-back, mellow, slow-moving atmosphere music"),
        ((0.45, 0.65), "medium energy, moderate pace, steady rhythm, balanced activity music"),
        ((0.65, 0.85), "high energy, active, driving rhythm, fast-paced, stimulating, busy arrangement music"),
        ((0.85, 1.0), "very high energy, hyperactive, restless, frantic, adrenaline-pumping, non-stop action music")
    ],

    'n_loudness': [
        ((0.0, 0.25), "very low loudness, barely audible, near silence, whisper-like volume, extremely quiet music"),
        ((0.25, 0.45), "low loudness, soft volume, background level, reduced amplitude, delicate sound music"),
        ((0.45, 0.70), "medium loudness, standard volume, normal mastering level music"),
        ((0.70, 0.85), "high loudness, loud volume, amplified sound, noisy, high amplitude music"),
        ((0.85, 1.0), "very high loudness, maximum volume, deafening, high decibels music")
    ],

    'speechiness': [
        # Speechiness > 0.66 to zazwyczaj podcasty, 0.33-0.66 to rap, < 0.33 to muzyka
        ((0.0, 0.33), "very low speechiness, purely musical track, no spoken words, fully melodic music"),
        ((0.33, 0.55), "low speechiness, mostly music with occasional spoken words or short background phrases"),
        ((0.55, 1.0), "medium to high speechiness, balanced mix of speech and music, frequent spoken segments, rap-like or talky structure"),
    ],

    # --- NOWOŚĆ: PRESETY AKTYWNOŚCI ---
    # Traktujemy je jak każdą inną cechę.
    # Wartość (Value) to nazwa presetu, którą potem rozpakujemy.
    # 'activity_preset': [
    #     ("deep_focus", "music for reading books, studying, learning, homework, deep focus, concentration, coding, programming, office work, brain power, do nauki, skupienie, praca, czytanie"),

    #     ("sleep_relax", "music for sleeping, falling asleep, insomnia, nap, meditation, yoga, mindfulness, spa, massage, calm down, anxiety relief, do spania, sen, relaks, medytacja, odpoczynek"),

    #     ("workout_intense", "music for gym, weightlifting, crossfit, boxing, cardio, running fast, sprint, beast mode, motivation, pump up, na siłownię, trening, bieganie, motywacja, ćwiczenia"),

    #     ("commute_jogging", "music for jogging, walking, commuting, driving, car ride, road trip, night drive, travel, jazda samochodem, do auta, podróż, spacer"),

    #     ("party_club", "music for party, house party, clubbing, dancing, dancefloor, celebration, drinking, festival, rave, disco, na imprezę, taniec, klubowa, domówka"),

    #     ("chores_background", "music for cleaning the house, cooking, kitchen, doing dishes, gardening, chores, morning coffee, sunday morning, sprzątanie, gotowanie, do tła, porządki"),

    #     ("sad_emotional", "music for feeling sad, crying, depression, lonely, heartbreak, breakup, melancholy, grieving, emotional, do płakania, smutek, depresja, doła"),

    #     ("romance_sexy", "music for date night, romantic dinner, candlelight, intimacy, cuddling, sex, sexy, seduction, bedroom, randka, seks, romantyczna, wieczór"),

    #     ("gaming", "music for gaming, playing games, esports, streaming, twitch, league of legends, cyberpunk, fps, rpg, do grania, gry, dla graczy")
    # ],

    'noise': [
        # Rzeczowniki (generyczne słowa)
        (None, "music song track playlist list recording audio sound genre style vibe type kind number piece"),

        # Czasowniki (związane z szukaniem)
        (None, "I am looking for I want I need search find play listen to give me recommend show me"),

        # Przymiotniki (fillery bez konkretnej treści)
        (None, "good very good nice great best cool amazing awesome some any kind of such a")
    ]
}

In [ ]:
!git add .
!git

In [221]:
ACTIVITY_RULES = {
    'deep_focus': [('instrumentalness', (0.8, 1.0)), ('speechiness', (0.0, 0.2)), ('energy', (0.1, 0.5))],
    'sleep_relax': [('energy', (0.0, 0.25)), ('n_tempo', (0.0, 0.35)), ('acousticness', (0.5, 1.0)), ('valence', (0.4, 0.7))],
    'workout_intense': [('energy', (0.8, 1.0)), ('n_tempo', (0.7, 1.0)), ('danceability', (0.5, 0.9))],
    'commute_jogging': [('energy', (0.5, 0.8)), ('n_tempo', (0.45, 0.7)), ('valence', (0.4, 0.9))],
    'party_club': [('danceability', (0.8, 1.0)), ('energy', (0.7, 1.0)), ('valence', (0.6, 1.0))],
    'chores_background': [('energy', (0.4, 0.7)), ('valence', (0.5, 0.9)), ('danceability', (0.4, 0.8))],
    'sad_emotional': [('valence', (0.0, 0.3)), ('energy', (0.0, 0.4))],
    'romance_sexy': [('n_tempo', (0.1, 0.5)), ('danceability', (0.5, 0.8)), ('energy', (0.2, 0.6))],
    'gaming': [('energy', (0.7, 1.0)), ('acousticness', (0.0, 0.2))]
}

In [222]:
# FEATURE_DESCRIPTIONS = {
#     'valence': [
#         (0.1, "very low valence, very sad, melancholic, dark, gloomy emotional mood music"),
#         (0.3, "low valence, bittersweet, thoughtful, introspective, moody emotional mood music"),
#         # (0.5, "medium valence, neutral emotional mood, neither clearly happy nor clearly sad music"),
#         (0.7, "high valence, positive, pleasant, warm, cheerful, uplifting emotional mood music"),
#         (0.9, "very high valence, very happy, joyful, exstatic, euphoric, bright, feel-good emotional mood music")
#     ],

#     'danceability': [
#         (0.1, "very low danceability, not danceable, abstract or experimental, weak or irregular rhythm music"),
#         (0.3, "low danceability, little groove, minimal rhythm, not primarily for dancing music"),
#         # (0.5, "medium danceability, some groove, steady rhythm music"),
#         (0.7, "high danceability, clear beat, strong groove, good for dancing, club-oriented music"),
#         (0.9, "very high danceability, strong groove, infectious rhythm, perfect for dancing, party, club banger music")
#     ],

#     'acousticness': [
#         (0.1, "very low acousticness, fully electronic, synthetic, digital, computer-generated sound music"),
#         (0.3, "low acousticness, mostly electronic with some subtle organic or acoustic elements music"),
#         # (0.5, "medium acousticness, balanced mix of acoustic and electronic instruments, hybrid sound music"),
#         (0.7, "high acousticness, mostly acoustic, organic, live instruments such as accoustic guitar or piano music"),
#         (0.9, "very high acousticness, fully acoustic, unplugged, natural, organic instruments only music")
#     ],

#     'n_tempo': [
#         (0.1, "very slow tempo, very slow pace, dragging rhythm music"),
#         (0.3, "slow tempo, downtempo, slow pace, relaxed rhythm music"),
#         (0.5, "medium tempo, moderate pace, walking pace music"),
#         (0.7, "fast tempo, uptempo, quick pace, energetic rhythm music"),
#         (0.9, "very fast tempo, rapid pace, racing rhythm, frantic speed music")
#     ],

#     'instrumentalness': [
#         (0.1, "very low instrumentalness, strong presence of vocals and lyrics, clear singing, vocal-focused track"),
#         (0.5, "medium instrumentalness, mix of vocals and instrumental sections, vocals present but not constant"),
#         (0.9, "very high instrumentalness, fully instrumental track, no vocals, no singing, music without lyrics")
#     ],

#     'energy': [
#         (0.1, "very low energy, motionless, static, sleep-inducing, minimal activity music"),
#         (0.3, "low energy, relaxed, laid-back, mellow, slow-moving atmosphere music"),
#         (0.5, "medium energy, moderate pace, steady rhythm, balanced activity music"),
#         (0.7, "high energy, active, driving rhythm, fast-paced, stimulating, busy arrangement music"),
#         (0.9, "very high energy, hyperactive, restless, frantic, adrenaline-pumping, non-stop action music")
#     ],

#     'n_loudness': [
#         (0.1, "very low loudness, barely audible, near silence, whisper-like volume, extremely quiet music"),
#         (0.3, "low loudness, soft volume, background level, reduced amplitude, delicate sound music"),
#         (0.5, "medium loudness, standard volume, normal mastering level music"),
#         (0.7, "high loudness, loud volume, amplified sound, noisy, high amplitude music"),
#         (0.9, "very high loudness, maximum volume, deafening, high decibels music")
#     ],

#     'speechiness': [
#         (0.1, "very low speechiness, purely musical track, no spoken words, fully melodic music"),
#         (0.3, "low speechiness, mostly music with occasional spoken words or short background phrases"),
#         (0.5, "medium speechiness, balanced mix of speech and music, frequent spoken segments, rap-like or talky structure"),
#     ],

#     'noise': [
#         # Rzeczowniki (generyczne słowa)
#         (None, "music song track playlist list recording audio sound genre style vibe type kind number piece"),
#         # TODO: przetestować, czy to się nie będzie myliło z instrumentallness

#         # Czasowniki (związane z szukaniem)
#         (None, "I am looking for I want I need search find play listen to give me recommend show me"),

#         # Przymiotniki (fillery bez konkretnej treści)
#         (None, "good very good nice great best cool amazing awesome some any kind of such a")
#     ]
# }

In [223]:
def prepare_feature_embeddings():
    """Embeduj wszystkie opisy jako 'passages' (raz przy starcie)"""

    feature_embeddings = {}

    for feature_name, descriptions in FEATURE_DESCRIPTIONS.items():
        # E5 wymaga "passage: " dla dokumentów
        passages = [f"passage: {desc}" for value, desc in descriptions]
        embeddings = model_e5.encode(passages, normalize_embeddings=True)
        feature_embeddings[feature_name] = embeddings

    return feature_embeddings

# Setup (raz)
feature_embeddings = prepare_feature_embeddings()

In [224]:
# def phrases_to_features(phrases_list, confidence_threshold=0.78, lang_code='pl'):
#     """
#     Mapuje listę fraz tekstowych na cechy audio, używając dwuetapowej klasyfikacji
#     opartej na osadzeniach (embeddings) modelu E5.

#     Logika działania:
#     1. Etap eliminacji: Sprawdza "surową" frazę (bez dopisku "muzyka") przeciwko wszystkim
#        kategoriom, w tym kategorii 'noise' (szum/śmieci). Jeśli fraza jest najbardziej
#        podobna do 'noise', zostaje odrzucona.
#     2. Etap precyzyjnego dopasowania: Dla fraz, które przeszły etap 1, dodawany jest
#        kontekst muzyczny (np. "muzyka" lub "music"). Następnie szukane jest najlepsze
#        dopasowanie do konkretnych cech audio (z pominięciem 'noise').

#     Funkcja korzysta ze zmiennych globalnych:
#         - model_e5: Model do generowania embeddingów (np. SentenceTransformer).
#         - FEATURE_DESCRIPTIONS: Słownik opisów cech.
#         - feature_embeddings: Wcześniej obliczone embeddingi dla opisów cech.

#     Args:
#         phrases_list (List[str]): Lista fraz do przeanalizowania (np. ["szybka", "rockowa"]).
#         confidence_threshold (float, optional): Minimalny próg podobieństwa cosinusowego,
#             powyżej którego dopasowanie jest uznawane za ważne. Domyślnie 0.78.
#         lang_code (str, optional): Kod języka ('pl' lub 'en') decydujący o sufiksie
#             kontekstowym ("muzyka" vs "music"). Domyślnie 'pl'.

#     Returns:
#         List[Tuple[str, Dict[str, Any]]]: Posortowana lista krotek (cecha, szczegóły),
#         uporządkowana malejąco według pewności (confidence).
#         Przykład:
#             [
#                 ('tempo', {'value': 'fast', 'confidence': 0.85}),
#                 ('genre', {'value': 'rock', 'confidence': 0.82})
#             ]
#     """
#     if not phrases_list:
#         return []

#     # ETAP 1: ELIMINACJA (Surowe frazy vs Wszystkie kategorie)

#     raw_queries = [f"query: {p}" for p in phrases_list]
#     raw_embeddings = model_e5.encode(raw_queries, normalize_embeddings=True)

#     valid_indices = [] # Indeksy fraz, które przeszły selekcję

#     for i, phrase in enumerate(phrases_list):
#         current_raw_emb = raw_embeddings[i].reshape(1, -1)

#         # Szukamy zwycięzcy dla surowej frazy
#         best_cat_raw = None
#         best_score_raw = -1.0

#         for feature_name, _ in FEATURE_DESCRIPTIONS.items():
#             target_embs = feature_embeddings[feature_name]

#             # Porównanie
#             sims = cosine_similarity(current_raw_emb, target_embs)[0]
#             max_score = float(np.max(sims))

#             if max_score > best_score_raw:
#                 best_score_raw = max_score
#                 best_cat_raw = feature_name

#         # decyzja czy wygrał 'noise'
#         if best_cat_raw == 'noise':
#             print(f" Odrzucono (wygrał Noise): '{phrase}' (score: {best_score_raw:.3f})")
#             continue # Fraza odpada
#         else:
#             # Fraza przeszła, zapamiętujemy jej indeks
#             valid_indices.append(i)

#     if not valid_indices:
#         return []

#     # ETAP 2: PRECYZYJNE DOPASOWANIE (z kontekstem "muzyka")

#     # Wyciągamy frazy, które przetrwały
#     clean_phrases = [phrases_list[i] for i in valid_indices]

#     # Dodajemy kontekst (suffix)
#     suffix = " muzyka" if lang_code == 'pl' else " music"
#     context_queries = [f"query: {suffix} {p}" for p in clean_phrases]
#     context_embeddings = model_e5.encode(context_queries, normalize_embeddings=True)

#     found_features = {}

#     for i, phrase in enumerate(clean_phrases):
#         current_emb = context_embeddings[i].reshape(1, -1)

#         phrase_best_cat = None
#         phrase_best_val = None
#         phrase_best_score = -1.0
#         phrase_best_desc = ""

#         # Iterujemy po cechach (już bez Noise, bo go wyeliminowaliśmy)
#         for feature_name, descriptions in FEATURE_DESCRIPTIONS.items():
#             if feature_name == 'noise':
#                 continue

#             target_embs = feature_embeddings[feature_name]
#             sims = cosine_similarity(current_emb, target_embs)[0]

#             max_idx = int(np.argmax(sims))
#             max_score = float(sims[max_idx])

#             if max_score > phrase_best_score:
#                 phrase_best_score = max_score
#                 phrase_best_cat = feature_name
#                 phrase_best_val = descriptions[max_idx][0]
#                 phrase_best_desc = descriptions[max_idx][1]

#         # Zapisujemy wynik (jeśli przekroczył próg jakości)
#         if phrase_best_score >= confidence_threshold:
#             print(f"   Fraza: '{phrase}' -> {phrase_best_cat} ({phrase_best_score:.3f})")

#             # Rozwiązywanie konfliktów (Max Pooling)
#             if phrase_best_cat in found_features:
#                 if phrase_best_score > found_features[phrase_best_cat]['confidence']:
#                     found_features[phrase_best_cat] = {
#                         'value': phrase_best_val,
#                         'confidence': phrase_best_score,
#                         # 'matched_description': phrase_best_desc
#                     }
#             else:
#                 found_features[phrase_best_cat] = {
#                     'value': phrase_best_val,
#                     'confidence': phrase_best_score,
#                     # 'matched_description': phrase_best_desc
#                 }

#     sorted_features = sorted(
#         found_features.items(),
#         key=lambda x: x[1]['confidence'],
#         reverse=True
#     )

#     return sorted_features

In [225]:
def phrases_to_features(phrases_list, confidence_threshold=0.78, lang_code='pl'):
    """
    Mapuje listę fraz tekstowych na cechy audio, używając dwuetapowej klasyfikacji
    opartej na osadzeniach (embeddings) modelu E5.

    Logika działania:
    1. Etap eliminacji: Sprawdza "surową" frazę (bez dopisku "muzyka") przeciwko wszystkim
       kategoriom, w tym kategorii 'noise' (szum/śmieci). Jeśli fraza jest najbardziej
       podobna do 'noise', zostaje odrzucona.
    2. Etap precyzyjnego dopasowania: Dla fraz, które przeszły etap 1, dodawany jest
       kontekst muzyczny (np. "muzyka" lub "music"). Następnie szukane jest najlepsze
       dopasowanie do konkretnych cech audio (z pominięciem 'noise').

    Funkcja korzysta ze zmiennych globalnych:
        - model_e5: Model do generowania embeddingów (np. SentenceTransformer).
        - FEATURE_DESCRIPTIONS: Słownik opisów cech.
        - feature_embeddings: Wcześniej obliczone embeddingi dla opisów cech.

    Args:
        phrases_list (List[str]): Lista fraz do przeanalizowania (np. ["szybka", "rockowa"]).
        confidence_threshold (float, optional): Minimalny próg podobieństwa cosinusowego,
            powyżej którego dopasowanie jest uznawane za ważne. Domyślnie 0.78.
        lang_code (str, optional): Kod języka ('pl' lub 'en') decydujący o sufiksie
            kontekstowym ("muzyka" vs "music"). Domyślnie 'pl'.

    Returns:
        List[Tuple[str, Dict[str, Any]]]: Posortowana lista krotek (cecha, szczegóły),
        uporządkowana malejąco według pewności (confidence).
        Przykład:
            [
                ('tempo', {'value': 'fast', 'confidence': 0.85}),
                ('genre', {'value': 'rock', 'confidence': 0.82})
            ]
    """
    if not phrases_list:
        return []

    # ETAP 1: ELIMINACJA (Surowe frazy vs Wszystkie kategorie)

    raw_queries = [f"query: {p}" for p in phrases_list]
    raw_embeddings = model_e5.encode(raw_queries, normalize_embeddings=True)

    valid_indices = [] # Indeksy fraz, które przeszły selekcję

    for i, phrase in enumerate(phrases_list):
        current_raw_emb = raw_embeddings[i].reshape(1, -1)

        # Szukamy zwycięzcy dla surowej frazy
        best_cat_raw = None
        best_score_raw = -1.0

        for feature_name, _ in FEATURE_DESCRIPTIONS.items():
            target_embs = feature_embeddings[feature_name]

            # Porównanie
            sims = cosine_similarity(current_raw_emb, target_embs)[0]
            max_score = float(np.max(sims))

            if max_score > best_score_raw:
                best_score_raw = max_score
                best_cat_raw = feature_name

        # decyzja czy wygrał 'noise'
        if best_cat_raw == 'noise':
            print(f" Odrzucono (wygrał Noise): '{phrase}' (score: {best_score_raw:.3f})")
            continue # Fraza odpada
        else:
            # Fraza przeszła, zapamiętujemy jej indeks
            valid_indices.append(i)

    if not valid_indices:
        return []

    # ETAP 2: PRECYZYJNE DOPASOWANIE (z kontekstem "muzyka")

    # Wyciągamy frazy, które przetrwały
    clean_phrases = [phrases_list[i] for i in valid_indices]

    # Dodajemy kontekst (suffix)
    suffix = " muzyka" if lang_code == 'pl' else " music"
    context_queries = [f"query: {suffix} {p}" for p in clean_phrases]
    context_embeddings = model_e5.encode(context_queries, normalize_embeddings=True)

    found_features = {}

    for i, phrase in enumerate(clean_phrases):
        current_emb = context_embeddings[i].reshape(1, -1)

        phrase_best_cat = None
        phrase_best_val = None
        phrase_best_score = -1.0
        phrase_best_desc = ""

        # Iterujemy po cechach (już bez Noise, bo go wyeliminowaliśmy)
        for feature_name, descriptions in FEATURE_DESCRIPTIONS.items():
            if feature_name == 'noise':
                continue

            target_embs = feature_embeddings[feature_name]
            sims = cosine_similarity(current_emb, target_embs)[0]

            max_idx = int(np.argmax(sims))
            max_score = float(sims[max_idx])

            if max_score > phrase_best_score:
                phrase_best_score = max_score
                phrase_best_cat = feature_name
                phrase_best_val = descriptions[max_idx][0]
                phrase_best_desc = descriptions[max_idx][1]

        # Zapisujemy wynik (jeśli przekroczył próg jakości)
        if phrase_best_score >= confidence_threshold:
            print(f"   Fraza: '{phrase}' -> {phrase_best_cat} ({phrase_best_score:.3f})")

            # Rozwiązywanie konfliktów (Max Pooling)
            if phrase_best_cat in found_features:
                if phrase_best_score > found_features[phrase_best_cat]['confidence']:
                    found_features[phrase_best_cat] = {
                        'value': phrase_best_val,
                        'confidence': phrase_best_score,
                        # 'matched_description': phrase_best_desc
                    }
            else:
                found_features[phrase_best_cat] = {
                    'value': phrase_best_val,
                    'confidence': phrase_best_score,
                    # 'matched_description': phrase_best_desc
                }

    sorted_features = sorted(
        found_features.items(),
        key=lambda x: x[1]['confidence'],
        reverse=True
    )

    return sorted_features

In [226]:
audio_features_queries = e5_queries_separated['AUDIO']
print(audio_features_queries)

try:
    lang_code = detect(picked_prompt)
except:
    lang_code = 'pl'

criteria_audio = phrases_to_features(audio_features_queries, lang_code=lang_code)
criteria_audio

['szybka']
   Fraza: 'szybka' -> n_tempo (0.832)


[('n_tempo', {'value': (0.8, 0.9), 'confidence': 0.8323066234588623})]

## Dopasowanie tagów

In [227]:
df_tag_embeddings = pd.read_parquet("../df_unique_tag_embeddings.parquet")
df_tag_embeddings

,tag,tag_embedding
0,cover,"[0.00831775926053524, 0.012594068422913551, -0..."
1,rnb,"[0.002914982382208109, 0.03465382382273674, -0..."
2,american,"[-0.0076368385925889015, 0.005218952428549528,..."
3,grunge,"[-0.010997523553669453, 0.02025771327316761, -..."
4,hip hop,"[0.024621689692139626, 0.01388248149305582, -0..."
...,...,...
95,swedish,"[0.014969518408179283, 0.014814469963312149, -..."
96,60s,"[0.002923240652307868, 0.019455421715974808, -..."
97,post hardcore,"[0.02131279744207859, 0.02616158500313759, -0...."
98,house,"[0.016028674319386482, 0.011981630697846413, -..."


Embeddingi tagów

In [228]:
TAG_VECS = np.array(df_tag_embeddings["tag_embedding"].to_list(), dtype=np.float32)
TAGS = df_tag_embeddings["tag"].tolist()

In [229]:
def map_phrases_to_tags(
    phrases: list[str],
    model,
    tag_vecs,
    tags_list,
    threshold: float = 0.65
) -> dict[str, float]:
    """
    Mapuje listę fraz z zapytania użytkownika na konkretne tagi z bazy danych.
    Stosuje strategię "Winner Takes All" (1 Fraza = 1 Najlepszy Tag).

    Dla każdej frazy (np. "szybki rock") oblicza podobieństwo do wszystkich dostępnych tagów
    i wybiera ten jedyny, który pasuje najlepiej (np. "Rock"), o ile przekracza próg pewności.

    Args:
        phrases (list[str]): Lista fraz wyekstrahowanych z promptu (np. przez GLiNER/SpaCy).
                             Przykład: ['rock', 'lata 90', 'gitara'].
        model: Model embeddingów (np. E5/SentenceTransformer), posiadający metodę `.encode()`.
               Musi być tym samym modelem, którym zwektoryzowano tagi.
        tag_vecs (np.ndarray): Macierz pre-kalkulowanych embeddingów tagów.
                               Wymiary: (Liczba_Tagów x Wymiar_Wektora).
        tags_list (list[str]): Lista nazw tagów odpowiadająca wierszom w `tag_vecs`.
                               Indeks i w `tags_list` musi odpowiadać wierszowi i w `tag_vecs`.
        threshold (float, optional): Minimalny próg podobieństwa (0.0 - 1.0).
                                     Jeśli najlepszy tag ma wynik niższy niż próg, fraza jest ignorowana.
                                     Domyślnie 0.65.

    Returns:
        dict[str, float]: Słownik znalezionych tagów i ich wag (podobieństwa).
                          Klucz: Nazwa taga (z `tags_list`).
                          Wartość: Score podobieństwa (0.0 - 1.0).
                          Jeśli kilka fraz wskazuje na ten sam tag, zachowywany jest najwyższy wynik.
    """
    if not phrases:
        return {}

    print(f"Mapowanie 1:1 dla fraz: {phrases}")

    # Batch Encoding (Szybkie wektoryzowanie zapytań)
    # Dodajemy prefix 'query:' zgodnie ze standardem E5
    q_vecs = model.encode(
        [f"query: {p}" for p in phrases],
        convert_to_numpy=True,
        normalize_embeddings=True
    ).astype("float32")

    # Obliczenie macierzy podobieństwa (Tags x Phrases)
    sims_matrix = cosine_similarity(tag_vecs, q_vecs)

    found_tags = {}

    # Iteracja po każdej frazie (kolumnie macierzy)
    for i, phrase in enumerate(phrases):
        # Pobieramy kolumnę wyników dla bieżącej frazy
        phrase_scores = sims_matrix[:, i]

        # Znajdujemy indeks najlepszego taga (argmax)
        best_idx = np.argmax(phrase_scores)
        best_score = float(phrase_scores[best_idx])
        best_tag_name = tags_list[best_idx]

        # Sprawdzamy, czy wynik przekaracza threshold
        if best_score >= threshold:
            print(f"    Fraza '{phrase}' -> Tag '{best_tag_name}' ({best_score:.3f})")

            # Zapisujemy wynik.
            # Jeśli inny tag już był zapisany (np. inna fraza też wskazała na "rock"),
            # bierzemy ten z wyższym scorem (Max Pooling)
            if best_tag_name in found_tags:
                found_tags[best_tag_name] = max(found_tags[best_tag_name], best_score)
            else:
                found_tags[best_tag_name] = best_score
        else:
            print(f"    Fraza '{phrase}' nie pasuje do żadnego tagu (max: {best_score:.3f})")
            pass

    return found_tags

In [230]:
tags_queries = e5_queries_separated['TAGS']
print(tags_queries)
criteria_tags = map_phrases_to_tags(tags_queries, model=model_e5, tag_vecs=TAG_VECS, tags_list=TAGS, threshold=0.70)
criteria_tags

[]


{}

## Algorytm

uzyskujemy wagi wsyztskich tagów zsumowane do 1

In [231]:
def get_query_tag_weights(
    raw_tags
) -> dict[str, float]:
    """
    Normalizuje wagi tak, by sumowały się do 1.
    """
    s = sum(raw_tags.values())
    if s <= 0:
        return raw_tags

    return {t: v / s for t, v in raw_tags.items()}

In [232]:
query_tag_weights = get_query_tag_weights(criteria_tags)
query_tag_weights

{}

In [233]:
def score_songs_by_tags(
    query_tag_weights: dict[str, float],
    inv_index: dict[str, np.ndarray],
    n_songs: int,
    idf_map: dict[str, float],
    use_idf: bool = True,
    query_pow: float = 1.0,
) -> np.ndarray:
    """
    Liczy dopasowanie (score) wszystkich utworów w bazie do tagów z zapytania.
    Działa w oparciu o szybki odwrócony indeks (Inverted Index).

    Algorytm sumuje wagi tagów znalezionych w piosence, opcjonalnie mnożąc je przez IDF
    (rzadkość tagu), a na końcu normalizuje wynik do zakresu 0.0 - 1.0.

    Args:
        query_tag_weights (dict[str, float]): Znormalizowane wagi tagów wyciągnięte z promptu.
            Klucz: nazwa tagu (np. 'rock'), Wartość: waga (np. 0.6).
        inv_index (dict[str, np.ndarray]): Odwrócony indeks bazy utworów.
            Klucz: nazwa tagu. Wartość: tablica indeksów (ID) piosenek, które mają ten tag.
            Umożliwia błyskawiczne wyszukiwanie bez iteracji po wszystkich piosenkach.
        song_tag_count (np.ndarray): Wektor o długości równej liczbie piosenek w bazie (`n_songs`).
            Służy tutaj do określenia wymiaru wektora wyników.
        idf_map (dict[str, float]): Mapa Inverse Document Frequency.
            Klucz: tag. Wartość: współczynnik rzadkości.
            Rzadkie tagi (np. 'harfa') mają wyższe IDF niż częste (np. 'pop'),
            co pozwala premiować unikalne dopasowania.
        use_idf (bool, optional): Czy uwzględniać rzadkość tagów przy scoringu.
            Domyślnie True. Zalecane, aby unikać faworyzowania bardzo popularnych gatunków.
        query_pow (float, optional): Wykładnik potęgi do wyostrzania wag z promptu.
            - 1.0: brak zmian (liniowo).
            - > 1.0 (np. 2.0): Zwiększa przepaść między ważnymi a mało ważnymi tagami z zapytania.
            Domyślnie 1.0.

    Returns:
        np.ndarray: Wektor o kształcie `(n_songs,)` z wartościami float w zakresie [0.0, 1.0].
            Indeks w wektorze odpowiada indeksowi piosenki w `df_songs`.
    """

    # Przetworzenie wag zapytania
    qtw = {}

    # Zmienna do trzymania maksymalnego możliwego wyniku
    max_theoretical_score = 0.0

    for t, w in query_tag_weights.items():
        w_processed = w ** query_pow

        # Pobieramy IDF dla tego taga
        this_idf = 1.0
        if use_idf and idf_map is not None:
            this_idf = float(idf_map.get(t, 1.0))

        # Zapisujemy finalną wagę punktową dla tego taga
        final_weight = w_processed * this_idf

        qtw[t] = final_weight

        # Dodajemy do mianownika: idealna piosenka miałaby wszystkie te tagi
        max_theoretical_score += final_weight

    # Inicjalizacja wektora wyników
    scores = np.zeros(n_songs, dtype=np.float32)

    # Sumowanie punktów (coverage)
    if max_theoretical_score > 0:
        for tag, weight in qtw.items():
            if tag not in inv_index:
                continue

            idxs = inv_index[tag]
            scores[idxs] += weight

        # normalizacja
        scores /= max_theoretical_score

    return np.clip(scores, 0.0, 1.0)

In [234]:
def build_idf_map(inv_index: dict[str, np.ndarray], n_songs: int) -> dict[str, float]:
    """
    Tworzy mapę IDF (Inverse Document Frequency) dla wszystkich tagów.

    Wzór: IDF = log(N / df)
    Gdzie:
      - N: całkowita liczba piosenek
      - df: liczba piosenek zawierających dany tag

    Tagi rzadkie (np. 'dark ambient') dostaną wysoki wynik (np. 3.5).
    Tagi pospolite (np. 'pop') dostaną niski wynik (np. 0.2).
    """
    idf_map = {}

    for tag, song_indices in inv_index.items():
        # df (Document Frequency) - w ilu piosenkach występuje ten tag
        df = len(song_indices)

        if df > 0:
            # Używamy logarytmu, żeby spłaszczyć wyniki
            idf_score = np.log10(n_songs / df)
            idf_map[tag] = float(idf_score)

    return idf_map

In [235]:
n_songs = len(df_songs)
IDF_MAP = build_idf_map(INV_INDEX, n_songs)

idf_list = list(IDF_MAP.items())
sorted_by_idf = sorted(idf_list, key=lambda item: item[1], reverse=True)
# sorted_by_idf

In [236]:
scores_songs_tag = score_songs_by_tags(
    query_tag_weights=query_tag_weights,
    inv_index=INV_INDEX,
    n_songs=len(df_songs),
    idf_map=None,
    use_idf=SCORING_CONFIG['use_idf'],
    query_pow=SCORING_CONFIG['query_pow']
)
scores_songs_tag

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [237]:
def retrieve_candidates_tags(
    scores_songs_tag: np.ndarray,
    n_candidates: int = 400,
    flat_delta: float = 0.05
):
    """
    Wybiera kandydatów z bazy utworów na podstawie obliczonych punktów (score) za tagi.
    Stosuje mechanizm "miękkiego odcięcia" (soft cut-off) – jeśli utwory tuż za progiem n_candidates
    mają bardzo zbliżony wynik, również są dołączane do listy.

    Funkcja polega na globalnej zmiennej:
        - df_songs (pd.DataFrame): Główna baza danych utworów.

    Args:
        scores_songs_tag (np.ndarray): Wektor wyników dopasowania tagów dla wszystkich utworów
            (musi mieć tę samą długość co df_songs).
        n_candidates (int, optional): Docelowa liczba kandydatów do zwrócenia. Domyślnie 400.
        flat_delta (float, optional): Margines tolerancji dla wyników. Utwory poza top N,
            których wynik różni się od ostatniego zaakceptowanego o mniej niż ta wartość,
            zostaną również dołączone. Domyślnie 0.05.

    Returns:
        pd.DataFrame: Podzbiór `df_songs` zawierający najlepszych kandydatów,
        z dodanymi kolumnami 'tag_score' oraz 'score'.
    """
    # bierzemy tylko te, które w ogóle coś dopasowały
    pos_mask = scores_songs_tag > 0.0
    if not np.any(pos_mask):
        return df_songs.iloc[0:0].copy()

    pos_idxs = np.where(pos_mask)[0]
    # sortujemy wszystkie dopasowane po score malejąco
    pos_idxs = pos_idxs[np.argsort(-scores_songs_tag[pos_idxs])]

    # jeśli i tak mniej niż n_candidates → bierzemy wszystkie
    if len(pos_idxs) <= n_candidates:
        final_idxs = pos_idxs
    else:
        # score na granicy top-n
        boundary_idx = n_candidates - 1
        boundary_score = float(scores_songs_tag[pos_idxs[boundary_idx]])

        # rozszerzamy cut tak długo, jak kolejne mają ten sam (w praktyce: bardzo podobny) score
        cut = n_candidates
        while (
            cut < len(pos_idxs)
            and abs(float(scores_songs_tag[pos_idxs[cut]]) - boundary_score) <= flat_delta
        ):
            cut += 1

        final_idxs = pos_idxs[:cut]

    candidates = df_songs.iloc[final_idxs].copy()
    candidates["tag_score"] = scores_songs_tag[final_idxs]
    candidates["score"] = candidates["tag_score"]  # spójność z resztą pipeline'u
    return candidates

In [238]:
def calculate_audio_match(candidates_df, audio_criteria):
    """
    Oblicza stopień dopasowania kandydatów do zadanych kryteriów audio (np. energy, danceability).
    Stosuje podejście iloczynowe: ocena końcowa to iloczyn dopasowań dla poszczególnych cech.

    Logika:
    1. Dla każdego kryterium obliczany jest dystans: |wartość_utworu - wartość_docelowa|.
    2. Dystans zamieniany jest na podobieństwo: 1.0 - dystans.
    3. Wyniki cząstkowe są mnożone (utwór musi być bliski wszystkim kryteriom naraz).

    Args:
        candidates_df (pd.DataFrame): DataFrame z utworami. Musi zawierać kolumny odpowiadające
            nazwom cech w audio_criteria (np. 'energy', 'tempo').
        audio_criteria (List[Tuple[str, Dict[str, float]]]): Lista krotek z kryteriami.
            Format: [('energy', {'value': 0.8, ...}), ('danceability', {'value': 0.5})].

    Returns:
        np.ndarray: Wektor o długości równej liczbie wierszy w candidates_df,
        zawierający wartości zmiennoprzecinkowe z zakresu 0.0 - 1.0.
    """
    if candidates_df.empty:
        return np.array([])

    # Jeśli nie ma kryteriów audio, zwracamy 1.0 (neutralny wpływ)
    # lub 0.0 (jeśli chcemy, by audio było wymagane - ale lepiej 1.0)
    if not audio_criteria:
        return np.ones(len(candidates_df))

    # Wektor końcowy (zaczynamy od samych jedynek)
    total_audio_score = np.ones(len(candidates_df))

    # Iterujemy po każdym kryterium (np. energy=0.9, danceability=0.7)
    for feature_name, criteria in audio_criteria:
        val_data = criteria['value']

        # ZABEZPIECZENIE:
        # Sprawdzamy czy w konfiguracji jest liczba (stara wersja) czy krotka/lista (nowa wersja)
        if isinstance(val_data, (list, tuple)):
            target_min, target_max = val_data
        else:
            # Jeśli user podał jedną liczbę, traktujemy to jako punkt (min=max)
            target_min = target_max = float(val_data)

        # Sprawdzamy, czy cecha istnieje w DataFrame
        if feature_name not in candidates_df.columns:
            continue

        # Pobieramy wartości z kolumny
        song_values = candidates_df[feature_name].to_numpy()

        # OBLICZANIE DYSTANSU (Similarity)
        # 1. Obliczamy o ile wartość poniżej min
        dist_below = np.maximum(0, target_min - song_values)

        # 2. Obliczamy o ile wartość jest powyżej max
        dist_above = np.maximum(0, song_values - target_max)

        # 3. Sumujemy dystans
        total_dist = dist_below + dist_above

        # 4. Zamieniamy na podobieństwo (1.0 - dystans)
        sim = np.clip(1.0 - total_dist, 0.0, 1.0)

        # Mnożymy (intersekcja warunków): Song musi spełniać Warunek 1 i Warunek 2
        total_audio_score *= sim

    return total_audio_score

In [239]:
def calculate_audio_match(candidates_df, audio_criteria):
    """
    Oblicza stopień dopasowania kandydatów do zadanych kryteriów audio (np. energy, danceability).
    Stosuje podejście iloczynowe: ocena końcowa to iloczyn dopasowań dla poszczególnych cech.

    Logika:
    1. Dla każdego kryterium obliczany jest dystans: |wartość_utworu - wartość_docelowa|.
    2. Dystans zamieniany jest na podobieństwo: 1.0 - dystans.
    3. Wyniki cząstkowe są mnożone (utwór musi być bliski wszystkim kryteriom naraz).

    Args:
        candidates_df (pd.DataFrame): DataFrame z utworami. Musi zawierać kolumny odpowiadające
            nazwom cech w audio_criteria (np. 'energy', 'tempo').
        audio_criteria (List[Tuple[str, Dict[str, float]]]): Lista krotek z kryteriami.
            Format: [('energy', {'value': 0.8, ...}), ('danceability', {'value': 0.5})].

    Returns:
        np.ndarray: Wektor o długości równej liczbie wierszy w candidates_df,
        zawierający wartości zmiennoprzecinkowe z zakresu 0.0 - 1.0.
    """
    if candidates_df.empty:
        return np.array([])

    if not audio_criteria:
        return np.ones(len(candidates_df))

    total_audio_score = np.ones(len(candidates_df))

    for feature_name, criteria in audio_criteria:
        val_data = criteria['value']

        # ZABEZPIECZENIE:
        # Sprawdzamy czy w konfiguracji jest liczba (stara wersja) czy krotka/lista (nowa wersja)
        if isinstance(val_data, (list, tuple)):
            target_min, target_max = val_data
        else:
            # Jeśli user podał jedną liczbę, traktujemy to jako punkt (min=max)
            target_min = target_max = float(val_data)

        if feature_name not in candidates_df.columns:
            continue

        song_values = candidates_df[feature_name].to_numpy()

        # === NOWA LOGIKA MATEMATYCZNA ===

        # 1. Obliczamy o ile wartość jest ZA MAŁA (poniżej min)
        # Jeśli piosenka ma 0.5, a min to 0.7 -> diff = 0.2
        # Jeśli piosenka ma 0.75, a min to 0.7 -> diff = -0.05 -> bierzemy 0 (bo maximum(0, ...))
        dist_below = np.maximum(0, target_min - song_values)

        # 2. Obliczamy o ile wartość jest ZA DUŻA (powyżej max)
        # Jeśli piosenka ma 0.9, a max to 0.8 -> diff = 0.1
        dist_above = np.maximum(0, song_values - target_max)

        # 3. Sumujemy dystans (zawsze jedna z tych wartości będzie zerem, albo obie jeśli trafiliśmy w środek)
        total_dist = dist_below + dist_above

        # 4. Zamieniamy na podobieństwo (1.0 - dystans)
        sim = np.clip(1.0 - total_dist, 0.0, 1.0)

        total_audio_score *= sim

    return total_audio_score

Fuzja - łączymy wyniki audio i tagów i robimy scoring

In [240]:
def merge_tag_and_audio_scores(df, audio_weight=0.3, use_tags=True):
    df = df.copy()
    # Jeśli nie ma tagów, waga Audio to 100% (1.0), inaczej bierzemy z configu
    w = audio_weight if use_tags else 1.0

    # Liczymy score (safe get zabezpiecza przed brakiem kolumn)
    df['score'] = (df.get('tag_score', 0) * (1 - w)) + (df.get('audio_score', 0) * w)

    # Uzupełniamy braki zerami tylko dla estetyki wyświetlania
    for col in ['tag_score', 'audio_score']:
        if col not in df: df[col] = 0.0

    return df.sort_values('score', ascending=False)

In [241]:
candidades_df = retrieve_candidates_tags(
    scores_songs_tag,
    n_candidates=RETRIEVAL_CONFIG['n_candidates'],
    flat_delta=RETRIEVAL_CONFIG['flat_delta']
)
candidades_df

,track_id,name,artist,spotify_preview_url,tags,genre,year,duration_ms,danceability,energy,...,spotify_url,explicit,album_images,spotify_id,n_tempo,n_loudness,tags_list,tags_count,tags_embedding,tag_count


In [242]:
has_tags = not candidades_df.empty

# Fallback
if not has_tags and criteria_audio:
    print("Brak wyników z tagów.")
    candidades_df = df_songs.copy()
elif has_tags:
    print(f"Znaleziono {len(candidades_df)} kandydatów po tagach.")
else:
    print("Pusto (brak tagów i brak audio).")



Brak wyników z tagów.


In [243]:
print(picked_prompt)

szybka


In [244]:
# Policz Audio Score
if not candidades_df.empty and criteria_audio:
    candidades_df['audio_score'] = calculate_audio_match(candidades_df, criteria_audio)

In [245]:
audio_match_vector = calculate_audio_match(candidades_df, criteria_audio)
candidades_df['audio_score'] = audio_match_vector
candidades_df[['artist', 'name', 'audio_score']].sample(3)

,artist,name,audio_score
12863,Tom Waits,Rosie,0.490839
16512,Borknagar,Inner Landscape,0.555838
22719,Matt Monro,From Russia With Love,0.565663


In [246]:
picked_prompt

'szybka'

In [247]:
candidades_merged_score_df = merge_tag_and_audio_scores(candidades_df, audio_weight=0.6, use_tags=has_tags)
candidades_merged_score_df[['artist', 'name', 'tag_score', 'audio_score', 'score']].head(20)

,artist,name,tag_score,audio_score,score
4508,Art Brut,Stand Down,0.0,1.0,1.0
4509,Acceptance,Permanent,0.0,1.0,1.0
49489,For Today,Nicodemus (The Seeker),0.0,1.0,1.0
49506,Breakdown of Sanity,Chapters,0.0,1.0,1.0
49531,Machinemade God,For Those Who Care,0.0,1.0,1.0
49607,Johnny Truant,Throne Vertigo,0.0,1.0,1.0
49610,Feed Her to the Sharks,Memory of You,0.0,1.0,1.0
49624,Greeley Estates,Loyal.com,0.0,1.0,1.0
4654,Mudvayne,Everything and Nothing,0.0,1.0,1.0
4657,Story of the Year,We Don't Care Anymore,0.0,1.0,1.0


Dzielimy zestaw kandydatów (piosenek) na trzy poziomy jakości (tzw. tiery) w zależności od wartości dopasowania (score).

Celem tej funkcji jest rozdzielenie wyników wyszukiwania według jakości dopasowania,
tak aby późniejsze etapy
mogły w różny sposób traktować utwory bardzo podobne, średnio podobne i słabe.

Utwory są przypisywane do trzech grup:
  * Tier A — wysokie dopasowanie (`score` >= `t_high`)
  * Tier B — średnie dopasowanie (`t_mid` <= `score` < `t_high`)
  * Tier C — niskie dopasowanie (`score` < `t_mid`)

In [248]:
def tier_by_score(candidates: pd.DataFrame, t_high: float, t_mid: float):
    """
    Dzieli kandydatów na trzy poziomy jakości (tiery) w zależności od wartości dopasowania (score).

    - Tier A: score >= t_high - bardzo dobre dopasowanie
    - Tier B: t_mid <= score < t_high - średnie dopasowanie
    - Tier C: score < t_mid - słabe dopasowanie

    Args:
        candidates (pd.DataFrame): Dane z kolumną 'score'.
        t_high (float): Próg dla wysokiego dopasowania.
        t_mid (float): Próg dla średniego dopasowania.

    Returns:
        tuple: (tier_a, tier_b, tier_c) — trzy DataFramey z podziałem wg jakości.
    """
    tier_a = candidates[candidates["score"] >= t_high].copy()
    tier_b = candidates[(candidates["score"] < t_high) & (candidates["score"] >= t_mid)].copy()
    tier_c = candidates[candidates["score"] < t_mid].copy()
    return tier_a, tier_b, tier_c

In [249]:
def calculate_dynamic_thresholds(candidates_df, high_threshold=0.75, mid_threshold=0.5):
    """
    Wylicza adaptacyjne progi punktowe (t_high, t_mid) służące do podziału wyników na Tiery (np. A i B).

    Logika łączy podejście względne (relative) z bezwzględnym (absolute):
    1. Podejście względne: Próg dostosowuje się do najlepszego wyniku w danej puli (max_score).
       Jeśli mamy świetne dopasowania (np. 0.95), próg Tier A rośnie (np. do 0.85), aby wybrać tylko "śmietankę".
    2. Podejście bezwzględne (Quality Gate): Próg nigdy nie spadnie poniżej ustalonego minimum (high_threshold).
       Jeśli najlepszy wynik to tylko 0.6, a high_threshold to 0.75, to t_high wyniesie 0.75.
       W efekcie Tier A pozostanie pusty, co jest pożądanym zachowaniem (nie pokazujemy słabych wyników jako "świetne").

    Args:
        candidates_df (pd.DataFrame): DataFrame z kandydatami, musi zawierać kolumnę 'score'.
        high_threshold (float, optional): Minimalna wartość bezwzględna dla Tier A. Domyślnie 0.75.
        mid_threshold (float, optional): Minimalna wartość bezwzględna dla Tier B. Domyślnie 0.5.

    Returns:
        Tuple[float, float]: Krotka zawierająca wyliczone progi (t_high, t_mid).
    """
    if candidates_df.empty:
        return 0.0, 0.0

    # Jaki jest absolutnie najlepszy wynik dla tego zapytania?
    max_score = candidates_df['score'].max()

    # LOGIKA ADAPTACYJNA

    # 1. Tier A: Musi być blisko lidera (np. max - 0.1),
    #    ale nie może być mniejszy niż 0.75.
    #    Jeśli max_score to 0.5, to t_high będzie 0.75 -> Tier A będzie pusty
    t_high = max(high_threshold, max_score - 0.1)

    # 2. Tier B: Musi być sensowny (np. max - 0.3),
    #    ALE nie mniejszy niż 0.4 (żeby nie brać śmieci).
    t_mid = max(mid_threshold, max_score - 0.2)

    return t_high, t_mid

In [250]:
t_high, t_mid = calculate_dynamic_thresholds(
    candidades_merged_score_df,
    high_threshold=WORKSET_CONFIG['min_absolute_high'],
    mid_threshold=WORKSET_CONFIG['min_absolute_mid']
)
t_high, t_mid

(0.9, 0.8)

In [251]:
tier_a, tier_b, tier_c = tier_by_score(
    candidades_merged_score_df,
    t_high,
    t_mid
)
print(len(tier_a), len(tier_b), len(tier_c))

5046 6644 38076


 Budowanie zbioru, z którego będziemy losować/wybierać utwory na podstawie dalszych cech (np. popularności).

Funkcja ma na celu stworzenie wystarczająco dużej i jakościowo sensownej puli kandydatów, z którymi będzie dalej pracować algorytm rekomendacji. Nie wybiera jeszcze finalnych piosenek, przygotowuje "roboczy" zestaw do późniejszego ważenia, filtrowania i losowania.


- min_final - mininalna wielkość zbioru, z którego potem będziemy czerpać.
- A - duży score
- B - średni
- C - niski

Logika:
* jeśli A >= min_final -> używamy tylko A
* elif A+B >= min_final -> używamy A + B
* else -> A + B + kawałek C

In [252]:
def build_working_set(
    tier_a: pd.DataFrame,
    tier_b: pd.DataFrame,
    tier_c: pd.DataFrame,
    target_pool_size: int,
    min_required_size: int,
    popularity_rescue_ratio: float = 0.2
) -> pd.DataFrame:
    """
    Buduje zbiór roboczy.
    Ważne: Przy dobieraniu z Tier B, dba o to, aby nie wziąć tylko niszowych utworów z dobrym score,
    ale też "uratować" popularne utwory, które mają przyzwoity score (są w Tier B).
    Args:
        tier_a (pd.DataFrame): Utwory o wysokim dopasowaniu (Hity jakościowe).
                               Bierzemy je zawsze w całości.
        tier_b (pd.DataFrame): Utwory o średnim dopasowaniu (Dobre, ale nie idealne).
                               Służą do uzupełnienia puli do `target_pool_size`.
        tier_c (pd.DataFrame): Utwory o niskim dopasowaniu (Słabe).
                               Używane tylko awaryjnie, jeśli nie osiągniemy `min_required_size`.
        target_pool_size (int): Docelowa wielkość zbioru roboczego (np. 50-100).
                                Tyle kandydatów chcemy dać algorytmowi losującemu,
                                aby miał swobodę w dobieraniu High/Mid/Low popularity.
        min_required_size (int): ABSOLUTNE MINIMUM. Tyle piosenek ma mieć finalna playlista.
                                 Jeśli A+B to za mało, dobierzemy stąd resztę z Tier C.
                                 Zalecane: tyle, ile user zażądał (np. 15).
        popularity_rescue_ratio (float): Jaka część slotów z Tieru B ma być zarezerwowana
                                         dla utworów NAJPOPULARNIEJSZYCH (zamiast tych z najlepszym score).
                                         Np. 0.3 oznacza, że 30% dobieranych utworów z B to hity,
                                         a 70% to najlepiej pasujące niszowe.
    """

    # 1. zawsze bierzemy całe Tier A
    working_parts = [tier_a]
    current_count = len(tier_a)

    # Jeśli mamy mniej kandydatów niż wynosi cel (target_pool_size), bierzemy z Tier B.
    if current_count < target_pool_size:
        needed = target_pool_size - current_count

        # Gdy Tier B jest mały, bierzemy całość
        if len(tier_b) <= needed:
            working_parts.append(tier_b)
            current_count += len(tier_b)
        # Gdy tier B jest duży, musimy wybierać.
        else:
            # Slot dla wysokich popularity
            n_pop = int(needed * popularity_rescue_ratio)
            # Slot dla najlepszego dopasowania semantycznego (Score)
            n_score = needed - n_pop

            # Wybieramy najpopularniejsze z dostępnych w Tier B
            b_pop_rescued = tier_b.sort_values("popularity", ascending=False).head(n_pop)
            # Usuwamy wybrane hity z puli, aby ich nie zduplikować
            remaining_b = tier_b.drop(b_pop_rescued.index)
            # Dobieramy resztę na podstawie czystego wyniku (score)
            b_score_top = remaining_b.sort_values("score", ascending=False).head(n_score)

            working_parts.extend([b_pop_rescued, b_score_top])
            current_count += (len(b_pop_rescued) + len(b_score_top))

    # Sprawdzamy, czy osiągnęliśmy absolutne minimum wymagane przez użytkownika (np. 15 piosenek na playlistę).
    # Jeśli nie, musimy sięgnąć po "słabsze" utwory, bo lepiej dać cokolwiek niż pustą playlistę.
    if current_count < min_required_size:
        needed = min_required_size - current_count
        part_c = tier_c.sort_values("score", ascending=False).head(needed)
        working_parts.append(part_c)

    # Zabezpieczenie na wypadek pustych wszystkich Tierów
    if not working_parts:
        return pd.DataFrame(columns=tier_a.columns)

    # Sklejamy
    working = pd.concat(working_parts, ignore_index=True)

    # Ustalamy po czym identyfikujemy piosenkę.
    subset_cols = []

    if 'id' in working.columns:
        subset_cols = ['id']
    elif 'name' in working.columns and 'artist' in working.columns:
        subset_cols = ['name', 'artist']

    if subset_cols:
        working = working.drop_duplicates(subset=subset_cols)
    else:
        pass

    # Finalne sortowanie
    working = working.sort_values("score", ascending=False).reset_index(drop=True)

    return working

In [253]:
working = build_working_set(
    tier_a, tier_b, tier_c,
    target_pool_size=WORKSET_CONFIG['target_pool_size'],
    min_required_size=WORKSET_CONFIG['min_required_size'],
    popularity_rescue_ratio=WORKSET_CONFIG['popularity_rescue_ratio']
)
# working
working["score"].value_counts()

,count
score,
1.000000,673
0.928492,7
0.928345,7
0.928362,6
0.911530,6
...,...
0.947006,1
0.946943,1
0.946939,1


Podział kandydatów wg popularności - chcemy uzyskać część popularnych utworów, część niszowych.

In [254]:
def bucket_by_popularity(working: pd.DataFrame, p_high: int, p_mid: int):
    """
    Dzieli utwory na trzy grupy (bucket'y) według ich popularności.

    - High:  popularity >= p_high
    - Mid:   p_mid <= popularity < p_high
    - Low:   popularity < p_mid

    Args:
        working (pd.DataFrame): Zbiór roboczy utworów z kolumną 'popularity'.
        p_high (int): Próg dla wysokiej popularności.
        p_mid (int): Próg dla średniej popularności.

    Returns:
        tuple: (pop_high, pop_mid, pop_low) — trzy DataFrame'y z podziałem wg popularności.
    """

    pop_high = working[working["popularity"] >= p_high].copy()
    pop_mid = working[(working["popularity"] < p_high) & (working["popularity"] >= p_mid)].copy()
    pop_low = working[working["popularity"] < p_mid].copy()
    return pop_high, pop_mid, pop_low

In [255]:
pop_high, pop_mid, pop_low = bucket_by_popularity(working, POPULARITY_CONFIG['p_high'], POPULARITY_CONFIG['p_mid'])
pop_high

,track_id,name,artist,spotify_preview_url,tags,genre,year,duration_ms,danceability,energy,...,spotify_id,n_tempo,n_loudness,tags_list,tags_count,tags_embedding,tag_count,audio_score,score,tag_score
24,TRXGDOA128F42712BF,Buttons,The Pussycat Dolls,https://p.scdn.co/mp3-preview/0584db756a9f1775...,"pop, female_vocalists, dance, hip_hop, rnb",Rock,2005,225560,0.548,0.825,...,314QzMOrpNUqWBUozYshf6,0.882752,0.873967,"[dance, female vocalists, hip hop, pop, rnb]",5,"[0.019436582922935486, 0.043213579803705215, -...",5,1.000000,1.000000,0.0
25,TROULHK128F4243EC0,99 Luftballons,Nena,https://p.scdn.co/mp3-preview/b08d7121360ab621...,"pop, female_vocalists, 80s, german, new_wave",None,2007,234800,0.404,0.433,...,2IJftBfq7pJ43tfnOR0RB3,0.806677,0.741020,"[80s, female vocalists, german, new wave, pop]",5,"[0.007905790582299232, 0.035911280661821365, -...",5,1.000000,1.000000,0.0
68,TRSASYZ12903D026BC,Somethin' Stupid,Frank Sinatra,https://p.scdn.co/mp3-preview/fcf85cd5f29beb41...,"jazz, american, 60s, oldies, love",None,2006,158933,0.281,0.413,...,4feXcsElKIVsGwkbnTHAfV,0.873275,0.796974,"[60s, american, jazz, love, oldies]",5,"[-0.010821688920259476, 0.017700262367725372, ...",5,1.000000,1.000000,0.0
185,TRMLNZN128F93167DC,Party Up,DMX,https://p.scdn.co/mp3-preview/c98280e4b8cb606c...,"rap, hip_hop",Rap,2009,276493,0.536,0.921,...,0LWkaEyQRkF0XAms8Bg1fC,0.844815,0.880315,"[hip hop, rap]",2,"[0.02882404811680317, 0.031562406569719315, -0...",2,1.000000,1.000000,0.0
236,TRJMBST12903CD9DDD,Chunky,Bruno Mars,https://p.scdn.co/mp3-preview/9a864a83d193e01d...,"80s, soul, funk, rnb",None,2016,186973,0.571,0.539,...,0mBKv9DkYfQHjdMcw2jdyI,0.845991,0.872914,"[80s, funk, rnb, soul]",4,"[0.015621482394635677, 0.02763506956398487, -0...",4,1.000000,1.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4911,TRUTNPC128F9300385,Clint Eastwood,Gorillaz,https://p.scdn.co/mp3-preview/20e27bbc6c7dd769...,"rock, electronic, alternative, indie, pop, alt...",None,2012,337720,0.659,0.712,...,1RKUoGiLEbcXN4GY4spQDx,0.703020,0.823324,"[00s, alternative, alternative rock, british, ...",11,"[-0.0017580949934199452, 0.0197638850659132, -...",11,0.903020,0.903020,0.0
4954,TROHCQJ128F9335BBC,Let's Get It On,Marvin Gaye,https://p.scdn.co/mp3-preview/de735bf3fe16dafd...,"soul, funk, 70s, oldies",None,2008,289920,0.517,0.738,...,627teoJpK7qZOxRRY8TNnv,0.702572,0.832799,"[70s, funk, oldies, soul]",4,"[0.0014989707851782441, 0.01777574233710766, -...",4,0.902572,0.902572,0.0
4967,TRIMAIK128F14AFBE5,All My Life,Foo Fighters,https://p.scdn.co/mp3-preview/982751373c269d03...,"rock, alternative, alternative_rock, hard_rock...",None,2009,263560,0.585,0.599,...,6tsojOQ5wHaIjKqIryLZK6,0.702309,0.853147,"[alternative, alternative rock, grunge, hard r...",5,"[-0.0024459518026560545, 0.03488243743777275, ...",5,0.902309,0.902309,0.0
4980,TROMUSF12903CAF896,Afraid,The Neighbourhood,https://p.scdn.co/mp3-preview/d5d9a119acf372dc...,"rock, alternative, indie, alternative_rock, in...",None,2013,251186,0.349,0.698,...,7brQHA2CgQpcMBiOlfiXYb,0.701785,0.853587,"[alternative, alternative rock, chill, indie, ...",6,"[0.001575007801875472, 0.037554699927568436, -...",6,0.901785,0.901785,0.0


In [256]:
def weighted_sample(df: pd.DataFrame, k: int, alpha: float):
    """
    Losuje `k` wierszy z DataFrame bez zwracania, faworyzując utwory z wyższym 'score'.
    Prawdopodobieństwo wylosowania utworu jest proporcjonalne do: (score ^ alpha).

    Rola parametru alpha (temperatura losowania):
    - alpha > 1 (np. 3.0): Strategia "Greedy" (chciwa). Bardzo mocno faworyzuje utwory z najwyższym wynikiem.
      Szansa na wylosowanie słabszych utworów drastycznie maleje.
    - alpha = 1.0: Prawdopodobieństwo liniowo zależne od wyniku.
    - alpha < 1 (np. 0.5): Spłaszczenie różnic. Słabsze utwory mają większą szansę zaistnieć.
    - alpha = 0: Losowanie jednostajne (każdy utwór ma taką samą szansę, ignorujemy score).

    Args:
        df (pd.DataFrame): Tabela z kandydatami (musi zawierać kolumnę 'score').
        k (int): Liczba utworów do wylosowania.
        alpha (float): Współczynnik skalowania wag (wykładnik potęgi).

    Returns:
        pd.DataFrame: Podzbiór `df` zawierający `k` wylosowanych wierszy.
    """
    if k <= 0 or len(df) == 0:
        return df.iloc[0:0]

    k = min(k, len(df))

    scores = df["score"].to_numpy(dtype=np.float32)
    scores = np.clip(scores, 1e-9, None)
    w = np.power(scores, alpha)
    w_sum = w.sum()
    if w_sum <= 0:
        w = np.full_like(w, 1.0 / len(w), dtype=np.float32)
    else:
        w /= w_sum

    idx = np.random.choice(len(df), size=k, replace=False, p=w)
    return df.iloc[idx]

In [257]:
def sample_final_songs(
    working: pd.DataFrame,
    popularity_cfg: dict,
    sampling_cfg: dict,
) -> pd.DataFrame:
    """
    Dokonuje finalnego wyboru utworów do playlisty z przygotowanej "puli roboczej" (working set).
    Stosuje zaawansowaną strategię mieszania popularności (Popularity Mix) oraz ważonego losowania (Weighted Sampling).

    Strategia:
    1. Forced Popular (Kotwice): Gwarantuje obecność znanych hitów (np. 2-3 utwory), aby użytkownik czuł się bezpiecznie.
       Stosuje losowanie ważone, aby nie grać w kółko tych samych 5 najpopularniejszych piosenek
    2. Popularity Buckets: Dzieli resztę miejsc wg proporcji (np. 40% High, 40% Mid, 20% Low/Discovery).
    3. Weighted Audio/Tag Score: Wewnątrz każdego bucketa utwory są losowane z prawdopodobieństwem zależnym od ich dopasowania
       do zapytania (score). Dzięki temu playlista trzyma klimat (audio/tagi), ale jest różnorodna.

    Args:
        working (pd.DataFrame): Pula kandydatów (zbiór roboczy) z kolumnami 'score' i 'popularity'.
        popularity_cfg (Dict[str, Any]): Konfiguracja popularności.
            - 'p_high', 'p_mid' (int): Progi punktowe podziału na buckety.
            - 'mix' (dict): Proporcje (np. {'high': 0.4, 'mid': 0.4, 'low': 0.2}).
            - 'forced_popular' (int): Liczba gwarantowanych hitów na start.
            - 'forced_popular_min' (int): Minimalna popularność, by utwór uznano za "hit" do sekcji forced.
        sampling_cfg (Dict[str, Any]): Konfiguracja losowania.
            - 'final_n' (int): Docelowa długość playlisty.
            - 'alpha' (float): "Temperatura" losowania. Im wyższa, tym bardziej faworyzujemy utwory z wysokim score.
            - 'shuffle' (bool): Czy przetasować finalną listę (True) czy zostawić posortowaną po score (False/Debug).

    Returns:
        pd.DataFrame: Gotowa playlista o długości `final_n` (lub mniejszej, jeśli brakło kandydatów).
    """

    if len(working) == 0:
        return working.iloc[0:0].copy()

    # Konfiguracja
    final_n = sampling_cfg.get("final_n", 15)
    alpha   = sampling_cfg.get("alpha", 2.0) # Siła wpływu score na losowanie

    p_high = popularity_cfg.get("p_high", 70)
    p_mid  = popularity_cfg.get("p_mid", 35)
    mix    = popularity_cfg.get("mix", {"high": 0.4, "mid": 0.35, "low": 0.25})

    forced_popular         = popularity_cfg.get("forced_popular", 0)
    forced_popular_min     = popularity_cfg.get("forced_popular_min", p_high)

    # 1. Bucketowanie po popularności
    pop_high, pop_mid, pop_low = bucket_by_popularity(working, p_high=p_high, p_mid=p_mid)

    final_parts = []

    # 2. Forced popular
    # Wybieramy pulę bardzo popularnych utworów (powyżej forced_popular_min)
    # Zazwyczaj to podzbiór pop_high
    forced_pool = working[working["popularity"] >= forced_popular_min].copy()

    # Zamiast brać top N na sztywno, losujemy z nich, preferując te z wyższym score.
    # Dzięki temu playlista jest bardziej różnorodna przy kolejnych wywołaniach.
    forced_taken = weighted_sample(forced_pool, forced_popular, alpha)

    final_parts.append(forced_taken)

    # Usuwamy wybrane utwory z bucketów, żeby ich nie dublować
    used_idx = set(forced_taken.index)
    pop_high = pop_high[~pop_high.index.isin(used_idx)]
    pop_mid  = pop_mid[~pop_mid.index.isin(used_idx)]
    pop_low  = pop_low[~pop_low.index.isin(used_idx)]

    # 3. Obliczamy ile slotów zostało do wypełnienia
    n_forced = len(forced_taken)
    remaining = max(0, final_n - n_forced)

    if remaining == 0:
        combined = pd.concat(final_parts, ignore_index=False)
        return combined.sort_values("score", ascending=False).reset_index(drop=True)

    # 4. Wyliczamy cele dla bucketów (Mix)
    target_high = int(round(remaining * mix.get("high", 0.0)))
    target_mid  = int(round(remaining * mix.get("mid",  0.0)))
    target_low  = remaining - target_high - target_mid  # Reszta do low

    # Clamp (nie możemy wziąć więcej niż jest w buckecie)
    target_high = min(target_high, len(pop_high))
    target_mid  = min(target_mid, len(pop_mid))
    target_low  = min(target_low, len(pop_low))

    # 5. Główne losowanie z bucketów (Weighted Sample wg Score)
    # Piosenki, które lepiej pasują (wyższy score), mają większą szansę na wylosowanie.
    sampled_high = weighted_sample(pop_high, target_high, alpha)
    sampled_mid  = weighted_sample(pop_mid,  target_mid,  alpha)
    sampled_low  = weighted_sample(pop_low,  target_low,  alpha)

    final_parts.extend([sampled_high, sampled_mid, sampled_low])

    combined = pd.concat(final_parts, ignore_index=False)

    # 6. Fill Gaps (Jeśli zaokrąglenia albo braki w bucketach sprawiły, że mamy za mało)
    if len(combined) < final_n:
        missing = final_n - len(combined)
        used_idx = set(combined.index)

        # Bierzemy resztki z całego working setu
        remaining_pool = working[~working.index.isin(used_idx)]
        extra = weighted_sample(remaining_pool, missing, alpha)

        combined = pd.concat([combined, extra], ignore_index=False)

    # 6. finalne tasowanie
    # frac=1 - "weź 100% wierszy, ale w losowej kolejności"
    do_shuffle = sampling_cfg.get("shuffle", True)
    if do_shuffle:
        combined = combined.sample(frac=1).reset_index(drop=True)
    else:
        # do debugowania: najlepsze na górze
        combined = combined.sort_values("score", ascending=False).reset_index(drop=True)

    return combined


In [258]:
picked_prompt

'szybka'

In [259]:
sample_final_songs(
    working,
    POPULARITY_CONFIG,
    SAMPLING_CONFIG
)[["name", "artist", "tags_list", "popularity", "score", "n_tempo"]]

,name,artist,tags_list,popularity,score,n_tempo
0,Cannibals,Mark Knopfler,"[country, rock]",48.0,0.924004,0.724004
1,Right Where I Belong,3 Doors Down,"[alternative rock, grunge, hard rock, rock]",25.0,0.903485,0.703485
2,Tempo Perdido,Legião Urbana,"[80s, pop, pop rock, rock]",75.0,0.979100,0.779100
3,Back Door Man,The Doors,"[60s, blues, blues rock, classic rock, psyched...",58.0,0.941887,0.741887
4,Come Undone,Duran Duran,"[80s, 90s, beautiful, british, new wave, pop]",76.0,0.928257,0.728257
5,Dilemma,Nelly,"[hip hop, love, rap, rnb]",81.0,0.903761,0.703761
6,Technicolour,Metrik,"[drum and bass, electronic]",8.0,0.933791,0.733791
7,Styrofoam,Fugazi,"[alternative, hardcore, indie, post hardcore, ...",35.0,0.958229,0.758229
8,Mota,The Offspring,"[90s, hard rock, punk, punk rock, rock]",51.0,0.955629,0.755629
9,Nail,Zola Jesus,"[alternative, ambient, dance, electronic, emo,...",5.0,0.943301,0.743301


In [260]:
# !git add .
# !git commit -m "improve config variables, comments"
# !git push